# Has Twenty20 changed the way test matches are played?
This code was written for a wionews.com article, ["Is T20 ruining Test cricket?"](http://www.wionews.com/cricket/analysis-is-t20-format-ruining-test-cricket-14150)

In [45]:
#The first part of the code involves scraping the data from cricketarchive.com
#this is just some libraries being imported that we will use later in scraping the website

from random import randint
import time
import requests

In [46]:
#get a list of years
list_orig = range(1990,2018)
list_orig

[1990,
 1991,
 1992,
 1993,
 .,
 .,
 .,
 2013,
 2014,
 2015,
 2016,
 2017]

In [47]:
#kind of looked at the url structure of the website, created this list because some of the urls contain these substrings
#and would help me later in scraping the website
#am yet to learn how to use full-blown scraping frameworks such as scrapy, will make that a next step

list_new = []
for i in range(0,len(list_orig)):
    try:
        m = str(list_orig[i])
        list_new.append(m)
        i_next = list_orig[i+1]
        j = str(i_next)
        k = j[-2:]
        l = m + "-" + k
        list_new.append(l)
    except IndexError:
        pass
print list_new

['1990', '1990-91', '1991', '1991-92', '1992', '1992-93', '1993', '1993-94', '1994', '1994-95', '1995', '1995-96', '1996', '1996-97', '1997', '1997-98', '1998', '1998-99', '1999', '1999-00', '2000', '2000-01', '2001', '2001-02', '2002', '2002-03', '2003', '2003-04', '2004', '2004-05', '2005', '2005-06', '2006', '2006-07', '2007', '2007-08', '2008', '2008-09', '2009', '2009-10', '2010', '2010-11', '2011', '2011-12', '2012', '2012-13', '2013', '2013-14', '2014', '2014-15', '2015', '2015-16', '2016', '2016-17', '2017']


In [48]:
#again more prep work creating urls before the actual scraping begins

test_urls = []
odi_urls = []
t20_urls = []

for year in list_new:
    a = "http://cricketarchive.com/Archive/Seasons/Lists/" + year + "_t_Match_List.html"
    test_urls.append(a)
    b = "http://cricketarchive.com/Archive/Seasons/Lists/" + year + "_o_Match_List.html"
    odi_urls.append(b)
    c = "http://cricketarchive.com/Archive/Seasons/Lists/" + year + "_itt_Match_List.html"
    t20_urls.append(c)

print test_urls

['http://cricketarchive.com/Archive/Seasons/Lists/1990_t_Match_List.html', 'http://cricketarchive.com/Archive/Seasons/Lists/1990-91_t_Match_List.html', 'http://cricketarchive.com/Archive/Seasons/Lists/1991_t_Match_List.html', 'http://cricketarchive.com/Archive/Seasons/Lists/1991-92_t_Match_List.html', '...',  'http://cricketarchive.com/Archive/Seasons/Lists/2015-16_t_Match_List.html', 'http://cricketarchive.com/Archive/Seasons/Lists/2016_t_Match_List.html', 'http://cricketarchive.com/Archive/Seasons/Lists/2016-17_t_Match_List.html', 'http://cricketarchive.com/Archive/Seasons/Lists/2017_t_Match_List.html']


In [49]:
print t20_urls

['http://cricketarchive.com/Archive/Seasons/Lists/1990_itt_Match_List.html', 'http://cricketarchive.com/Archive/Seasons/Lists/1990-91_itt_Match_List.html', 'http://cricketarchive.com/Archive/Seasons/Lists/1991_itt_Match_List.html', 'http://cricketarchive.com/Archive/Seasons/Lists/1991-92_itt_Match_List.html', '....', 'http://cricketarchive.com/Archive/Seasons/Lists/2014-15_itt_Match_List.html', 'http://cricketarchive.com/Archive/Seasons/Lists/2015_itt_Match_List.html', 'http://cricketarchive.com/Archive/Seasons/Lists/2015-16_itt_Match_List.html', 'http://cricketarchive.com/Archive/Seasons/Lists/2016_itt_Match_List.html', 'http://cricketarchive.com/Archive/Seasons/Lists/2016-17_itt_Match_List.html', 'http://cricketarchive.com/Archive/Seasons/Lists/2017_itt_Match_List.html']


In [1]:
#ok so this is where some of the scraping work begins
#most of the code below taken from https://kazuar.github.io/scraping-tutorial/ and
#https://brennan.io/2016/03/02/logging-in-with-requests/
#put my username and password in a text file, the code picks it up from there, logs in to the website
# creates a session that we'll be using for the rest of the time we're scraping data


import requests
import re

session_requests = requests.session()
login_url = "https://my.cricketarchive.com/"
result = session_requests.get(login_url)
sourcez = result.text


csrf_hash2 =str((re.findall(r"var csrfHash = '(.*)';", sourcez))[0]) 

f = open('passwords_ignore.txt',"r")
lines = f.readlines()
emailid = lines[0].strip()
passwd = lines[1].strip()

payload = {"email": emailid,"password": passwd,"vo-action": "login","csrfHash": csrf_hash2}

print csrf_hash2

response = session_requests.post(login_url,data = payload,headers = dict(referer=login_url))

#r = session_requests.get('http://cricketarchive.com/Archive/Seasons/Lists/2016_itt_Match_List.html')
#print r.text
#this is to test if we've logged in


6853f9f51f302ae418c8c40f588b1a34


In [ ]:
r = session_requests.get('http://cricketarchive.com/Archive/Seasons/Lists/2016_itt_Match_List.html')
print r.text
#this is to test if we've logged in

In [79]:
#this creates folders where we'll be downloading the initial set of pages
#this initial set of pages will then be used to generate the actual set of pages with data 
#on each match and its scoreboard

import os

folders = ["test","odi","t20"]
for folder in folders:
    if not os.path.exists(folder):
        os.makedirs(folder)

In [77]:
r = session_requests.get('http://cricketarchive.com/Archive/Seasons/Lists/1990_itt_Match_List.html')
print r.status_code
#r.raise_for_status()
#this is to test what status code we're getting for pages that don't really exist

404


In [78]:
#just some checking
print len(odi_urls)
print len(test_urls)
print len(t20_urls)

55
55
55


In [80]:
#what this code does is download a page for each cricket season, each of these 'season' pages have links to pages for 
#the various ODI, T20 and Test matches played in that season

import time
import re
import os

for url in t20_urls:
    print 'url being processed: ' + url
    responsex = session_requests.get(url)
    if responsex.status_code == 404:
        print '404 error here'
        time.sleep(10)
        continue
    else:
        try:
            last = (re.findall(r"http://cricketarchive.com/Archive/Seasons/Lists/(.*)", url))[0]
            filenamey = 't20/' + last
            fnamez = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenamey)
            with open(fnamez, "w") as filex:
                filex.write(responsex.content)
            print url + ' successfully written'
            delay = randint(10,15)
            time.sleep(delay)
        except:
            print "Something went wrong here."
print 't20s' + ' done'

url being processed: http://cricketarchive.com/Archive/Seasons/Lists/1990_itt_Match_List.html
404 error here
url being processed: http://cricketarchive.com/Archive/Seasons/Lists/1990-91_itt_Match_List.html
404 error here
url being processed: http://cricketarchive.com/Archive/Seasons/Lists/1991_itt_Match_List.html
404 error here
.
.
.
url being processed: http://cricketarchive.com/Archive/Seasons/Lists/2016-17_itt_Match_List.html
http://cricketarchive.com/Archive/Seasons/Lists/2016-17_itt_Match_List.html successfully written
url being processed: http://cricketarchive.com/Archive/Seasons/Lists/2017_itt_Match_List.html
404 error here
t20s done


In [81]:
#this gets all the match lists for ODIs

for url in odi_urls:
    print 'url being processed: ' + url
    responsex = session_requests.get(url)
    if responsex.status_code == 404:
        print '404 error here'
        time.sleep(10)
        continue
    else:
        try:
            last = (re.findall(r"http://cricketarchive.com/Archive/Seasons/Lists/(.*)", url))[0]
            filenamey = 'odi/' + last
            fnamez = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenamey)
            with open(fnamez, "w") as filex:
                filex.write(responsex.content)
            print url + ' successfully written'
            delay = randint(10,15)
            time.sleep(delay)
        except:
            print "Something went wrong here."
print 'odis' + ' done'

url being processed: http://cricketarchive.com/Archive/Seasons/Lists/1990_o_Match_List.html
http://cricketarchive.com/Archive/Seasons/Lists/1990_o_Match_List.html successfully written
url being processed: http://cricketarchive.com/Archive/Seasons/Lists/1990-91_o_Match_List.html
http://cricketarchive.com/Archive/Seasons/Lists/1990-91_o_Match_List.html successfully written
.
.
.
url being processed: http://cricketarchive.com/Archive/Seasons/Lists/2016-17_o_Match_List.html
http://cricketarchive.com/Archive/Seasons/Lists/2016-17_o_Match_List.html successfully written
url being processed: http://cricketarchive.com/Archive/Seasons/Lists/2017_o_Match_List.html
404 error here
odis done


In [82]:
#this gets all the match lists for tests

for url in test_urls:
    print 'url being processed: ' + url
    responsex = session_requests.get(url)
    if responsex.status_code == 404:
        print '404 error here'
        time.sleep(10)
        continue
    else:
        try:
            last = (re.findall(r"http://cricketarchive.com/Archive/Seasons/Lists/(.*)", url))[0]
            filenamey = 'test/' + last
            fnamez = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenamey)
            with open(fnamez, "w") as filex:
                filex.write(responsex.content)
            print url + ' successfully written'
            #delay = randint(10,15)
            time.sleep(10)
        except:
            print "Something went wrong here."
print 'tests' + ' done'

url being processed: http://cricketarchive.com/Archive/Seasons/Lists/1990_t_Match_List.html
http://cricketarchive.com/Archive/Seasons/Lists/1990_t_Match_List.html successfully written
url being processed: http://cricketarchive.com/Archive/Seasons/Lists/1990-91_t_Match_List.html
http://cricketarchive.com/Archive/Seasons/Lists/1990-91_t_Match_List.html successfully written
.
.
.
url being processed: http://cricketarchive.com/Archive/Seasons/Lists/1995-96_t_Match_List.html
http://cricketarchive.com/Archive/Seasons/Lists/1995-96_t_Match_List.html successfully written
url being processed: http://cricketarchive.com/Archive/Seasons/Lists/1996_t_Match_List.html


ConnectionError: HTTPConnectionPool(host='cricketarchive.com', port=80): Max retries exceeded with url: /Archive/Seasons/Lists/1996_t_Match_List.html (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x7f637ec62350>: Failed to establish a new connection: [Errno 110] Connection timed out',))

In [83]:
test_urls.index("http://cricketarchive.com/Archive/Seasons/Lists/1996_t_Match_List.html")

12

In [85]:
#the previous code block stopped midway, this resumes the scraping for the test match lists

for url in test_urls[12:]:
    print 'url being processed: ' + url
    responsex = session_requests.get(url)
    if responsex.status_code == 404:
        print '404 error here'
        time.sleep(10)
        continue
    else:
        try:
            last = (re.findall(r"http://cricketarchive.com/Archive/Seasons/Lists/(.*)", url))[0]
            filenamey = 'test/' + last
            fnamez = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenamey)
            with open(fnamez, "w") as filex:
                filex.write(responsex.content)
            print url + ' successfully written'
            delay = randint(10,15)
            time.sleep(10)
        except:
            print "Something went wrong here."
print 'tests' + ' done'

url being processed: http://cricketarchive.com/Archive/Seasons/Lists/1996_t_Match_List.html
http://cricketarchive.com/Archive/Seasons/Lists/1996_t_Match_List.html successfully written
url being processed: http://cricketarchive.com/Archive/Seasons/Lists/1996-97_t_Match_List.html
http://cricketarchive.com/Archive/Seasons/Lists/1996-97_t_Match_List.html successfully written
.
.
.
url being processed: http://cricketarchive.com/Archive/Seasons/Lists/2016-17_t_Match_List.html
http://cricketarchive.com/Archive/Seasons/Lists/2016-17_t_Match_List.html successfully written
url being processed: http://cricketarchive.com/Archive/Seasons/Lists/2017_t_Match_List.html
404 error here
tests done


In [86]:
#this creates the folders into which the scoreboards for tests, ODIs and T20 internationals will be downloaded

folders = ["test_cards","odi_cards","t20_cards"]
for folder in folders:
    if not os.path.exists(folder):
        os.makedirs(folder)

In [87]:
os.listdir("t20")

['2004-05_itt_Match_List.html',
 '2005-06_itt_Match_List.html',
 '2005_itt_Match_List.html',
 '2006-07_itt_Match_List.html',
 '2006_itt_Match_List.html',
 '2007-08_itt_Match_List.html',
 '2007_itt_Match_List.html',
 '2008-09_itt_Match_List.html',
 '2008_itt_Match_List.html',
 '2009-10_itt_Match_List.html',
 '2009_itt_Match_List.html',
 '2010-11_itt_Match_List.html',
 '2010_itt_Match_List.html',
 '2011-12_itt_Match_List.html',
 '2011_itt_Match_List.html',
 '2012-13_itt_Match_List.html',
 '2012_itt_Match_List.html',
 '2013-14_itt_Match_List.html',
 '2013_itt_Match_List.html',
 '2014-15_itt_Match_List.html',
 '2014_itt_Match_List.html',
 '2015-16_itt_Match_List.html',
 '2015_itt_Match_List.html',
 '2016-17_itt_Match_List.html',
 '2016_itt_Match_List.html']

In [89]:
#this allows us to scrape the urls for each match scoreboard

from lxml import html
import csv
import os

headings = ['format','number','date','year','team1','team2','ground','url']
with open("url_store.csv", "w") as filem:   
    wr = csv.writer(filem, delimiter = ',' , quotechar = '"' )
    wr.writerow(headings)

for i in ["test","odi","t20"]:
    d = 0
    print "starting" + i
    for j in os.listdir(i):
        filex = i + "/" + j
        print "processing " + filex
        filey =  os.path.join(os.path.dirname(os.path.realpath('__file__')), filex)
        with open(filey) as filez:
            sourcex = filez.read()
            treex = html.document_fromstring(sourcex)
            nodes = treex.xpath("//td[@valign='top']/a[contains(@href,'/Scorecards/')]")
            for node in nodes:
                row_list = []
                format = i
                row_list.append(format) #hmm, noticed this later, seems format is a reserved word, didn't give me errors though
                d += 1
                zrs = str(d).zfill(4)
                number = i + "_" + zrs
                row_list.append(number)
                date = node.xpath("preceding::td[2]/text()")[0]
                row_list.append(date)
                row_list.append(date[-4:])
                opponents = node.xpath("text()")[0]
                opp_list = opponents.split(' v ')
                row_list.append(opp_list[0])
                row_list.append(opp_list[1])
                ground = node.xpath("following::td[1]/a/text()")[0]
                row_list.append(ground)
                url = node.xpath("@href")[0]
                row_list.append(url)
                with open("url_store.csv", "a") as filen:
                    wrz = csv.writer(filen, delimiter = ',' , quotechar = '"' )
                    wrz.writerow(row_list)
        print "finished " + filex
    print "finished " + i

filen.close()
print 'done and done'

startingtest
processing test/1997_t_Match_List.html
finished test/1997_t_Match_List.html
processing test/2005_t_Match_List.html
finished test/2005_t_Match_List.html
.
.
.
processing test/2016_t_Match_List.html
finished test/2016_t_Match_List.html
finished test
startingodi
processing odi/1997_o_Match_List.html
finished odi/1997_o_Match_List.html
.
.
.
processing t20/2016_itt_Match_List.html
finished t20/2016_itt_Match_List.html
finished t20
done and done


In [95]:
#this generates the list of urls to each scoreboard, be it ODI, T20 or Test

import pandas as pd

df = pd.read_csv('url_store.csv')

for index, row in df.iterrows():
    print "processing " + row['number']
    urlx = "http://cricketarchive.com" + row['url']
    responseq = session_requests.get(urlx)
    filenamey = row['format'] + "_cards/" + row['number'] + ".html"
    fnamez = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenamey)
    with open(fnamez, "w") as filex:
        filex.write(responseq.content)
    print "finished " + row['number']
    delay = randint(10,15)
    time.sleep(delay)

print 'finished dloading cards'

processing test_0001
finished test_0001
processing test_0002
finished test_0002
processing test_0003
finished test_0003
processing test_0004
finished test_0004
.
.
.
processing t20_0616
finished t20_0616
processing t20_0617
finished t20_0617
finished dloading cards


In [20]:
#I'll be honest, some of the code blocks below are just pure nonsense
# I was trying to figure out a way to find the country a ground is located
#that information wasn't really in the first set of pages I downloaded
# so I had to download the webpage for each ground, where there is an entry mentioning the country the ground
#is located in, scrape that entry and put that information back in the original csv file


import pandas as pd

df = pd.read_csv('url_store.csv')

In [21]:
df.head()

,format,number,date,year,team1,team2,ground,url
0,test,test_0001,05 Jun 1997,1997,England,Australia,"Edgbaston, Birmingham",/Archive/Scorecards/64/64098.html
1,test,test_0002,19 Jun 1997,1997,England,Australia,"Lord's Cricket Ground, St John's Wood",/Archive/Scorecards/64/64171.html
2,test,test_0003,03 Jul 1997,1997,England,Australia,"Old Trafford, Manchester",/Archive/Scorecards/64/64259.html
3,test,test_0004,24 Jul 1997,1997,England,Australia,"Headingley, Leeds",/Archive/Scorecards/64/64378.html
4,test,test_0005,02 Aug 1997,1997,Sri Lanka,India,"R Premadasa Stadium, Colombo",/Archive/Scorecards/64/64422.html


In [31]:
#this gets  list of all the grounds at which matches -- ODI, T20 and Test -- have taken place over the past 25 years

import numpy

#finding uniques in a column
df['ground'].unique()

array(['Edgbaston, Birmingham', "Lord's Cricket Ground, St John's Wood",
       'Old Trafford, Manchester', 'Headingley, Leeds',
       'R Premadasa Stadium, Colombo', 'Trent Bridge, Nottingham',
       'Sinhalese Sports Club Ground, Colombo',
       "The Foster's Oval, Kennington",
       'Riverside Ground, Chester-le-Street', 'Asgiriya Stadium, Kandy',
       'The Brit Oval, Kennington', 'National Stadium, Karachi',
.
.
.
       'Greater Noida Sports Complex Ground, Greater Noida',
       'Moses Mabhida Stadium, Durban',
       'Central Broward Regional Park Stadium Turf Ground, Lauderhill',
       'ANZ Stadium, Sydney', 'Sylhet Stadium, Sylhet',
       'Wanderers Cricket Ground, Windhoek', 'Magheramason, Bready',
       'Simonds Stadium, Geelong'], dtype=object)

In [39]:
#this code generates a list of urls, urls for pages on each of the grounds, this page will have the country information
#that we will scrape later. 

from lxml import html
import csv
import os

headings = ['ground','ground_url']
with open("ground_urls.csv", "w") as filem:   
    wr = csv.writer(filem, delimiter = ',' , quotechar = '"' )
    wr.writerow(headings)

for i in ["test","odi","t20"]:
    d = 0
    print "starting" + i
    for j in os.listdir(i):
        filex = i + "/" + j
        print "processing " + filex
        filey =  os.path.join(os.path.dirname(os.path.realpath('__file__')), filex)
        with open(filey) as filez:
            sourcex = filez.read()
            treex = html.document_fromstring(sourcex)
            nodes = treex.xpath("//td[@valign='top']/a[contains(@href,'/Scorecards/')]")
            for node in nodes:
                row_list = []
                ground = node.xpath("following::td[1]/a/text()")[0]
                row_list.append(ground)
                ground_url = node.xpath("following::td[1]/a/@href")[0]
                row_list.append(ground_url)
                with open("ground_urls.csv", "a") as filen:
                    wrz = csv.writer(filen, delimiter = ',' , quotechar = '"' )
                    wrz.writerow(row_list)
        print "finished " + filex
    print "finished " + i

filen.close()
print 'done and done'

startingtest
processing test/1997_t_Match_List.html
finished test/1997_t_Match_List.html
processing test/2005_t_Match_List.html
finished test/2005_t_Match_List.html
.
.
.
processing t20/2016_itt_Match_List.html
finished t20/2016_itt_Match_List.html
finished t20
done and done


In [20]:
import pandas as pd

In [21]:
df = pd.read_csv('ground_urls.csv')

In [22]:
df.head(n=5)

,ground,ground_url
0,"Edgbaston, Birmingham",/Archive/Grounds/11/321.html
1,"Lord's Cricket Ground, St John's Wood",/Archive/Grounds/11/596.html
2,"Old Trafford, Manchester",/Archive/Grounds/11/618.html
3,"Headingley, Leeds",/Archive/Grounds/11/570.html
4,"R Premadasa Stadium, Colombo",/Archive/Grounds/25/1707.html


In [23]:
df['complete_url'] = "http://cricketarchive.com" + df['ground_url'].astype(str)

In [24]:
df['complete_url'][1]

'http://cricketarchive.com/Archive/Grounds/11/596.html'

In [25]:
df.head(n=5)

,ground,ground_url,complete_url
0,"Edgbaston, Birmingham",/Archive/Grounds/11/321.html,http://cricketarchive.com/Archive/Grounds/11/3...
1,"Lord's Cricket Ground, St John's Wood",/Archive/Grounds/11/596.html,http://cricketarchive.com/Archive/Grounds/11/5...
2,"Old Trafford, Manchester",/Archive/Grounds/11/618.html,http://cricketarchive.com/Archive/Grounds/11/6...
3,"Headingley, Leeds",/Archive/Grounds/11/570.html,http://cricketarchive.com/Archive/Grounds/11/5...
4,"R Premadasa Stadium, Colombo",/Archive/Grounds/25/1707.html,http://cricketarchive.com/Archive/Grounds/25/1...


In [26]:
new_urls = df['complete_url'].unique()
new_urls

array(['http://cricketarchive.com/Archive/Grounds/11/321.html',
       'http://cricketarchive.com/Archive/Grounds/11/596.html',
       'http://cricketarchive.com/Archive/Grounds/11/618.html',
       'http://cricketarchive.com/Archive/Grounds/11/570.html',
       'http://cricketarchive.com/Archive/Grounds/25/1707.html',
       'http://cricketarchive.com/Archive/Grounds/11/664.html',
       'http://cricketarchive.com/Archive/Grounds/25/1708.html',
       'http://cricketarchive.com/Archive/Grounds/11/594.html',
       'http://cricketarchive.com/Archive/Grounds/11/407.html',
       'http://cricketarchive.com/Archive/Grounds/25/1719.html',
.
.
.
       'http://cricketarchive.com/Archive/Grounds/19/1344.html',
       'http://cricketarchive.com/Archive/Grounds/60/1297.html',
       'http://cricketarchive.com/Archive/Grounds/2/83.html'], dtype=object)

In [27]:
#the array is just changed to a list, python terms, don't worry about it

new_list = new_urls.tolist()
new_list

['http://cricketarchive.com/Archive/Grounds/11/321.html',
 'http://cricketarchive.com/Archive/Grounds/11/596.html',
 'http://cricketarchive.com/Archive/Grounds/11/618.html',
.
.
.
 'http://cricketarchive.com/Archive/Grounds/4/246.html',
 'http://cricketarchive.com/Archive/Grounds/19/1344.html',
 'http://cricketarchive.com/Archive/Grounds/60/1297.html',
 'http://cricketarchive.com/Archive/Grounds/2/83.html']

In [28]:
#this logs in to the website again
#the list of urls we've generated for each of the grounds will now be opened and the pages downloaded

import requests
import re

session_requests = requests.session()
login_url = "https://my.cricketarchive.com/"
result = session_requests.get(login_url)
sourcez = result.text

#most of the code below taken from https://kazuar.github.io/scraping-tutorial/ and
#https://brennan.io/2016/03/02/logging-in-with-requests/

csrf_hash2 =str((re.findall(r"var csrfHash = '(.*)';", sourcez))[0]) 

f = open('passwords_ignore.txt',"r")
lines = f.readlines()
emailid = lines[0].strip()
passwd = lines[1].strip()

payload = {"email": emailid,"password": passwd,"vo-action": "login","csrfHash": csrf_hash2}

print csrf_hash2

response = session_requests.post(login_url,data = payload,headers = dict(referer=login_url))

#r = session_requests.get('http://cricketarchive.com/Archive/Seasons/Lists/2016_itt_Match_List.html')
#print r.text
#this is to test if we've logged in


18dd971b52950cc851836d0ed276001c


In [134]:
import os

os.makedirs('grounds')

In [3]:
#this is just test code to see if we can scrape the country field from the page for each ground

from lxml import html
import os

country_list = []
filenamey = 'grounds/001.html'
fnamez = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenamey)
with open(fnamez) as filez:
    sourcex = filez.read()
#print sourcex
treex = html.document_fromstring(sourcex)
#print(html.tostring(treex, pretty_print=True))
country = treex.xpath('//b[text()="Country:"]/following::td[1]/text()')[0]
#country = treex.xpath('//*[@id="columnLeft"]/table/tbody/tr[3]/td[2]/text()')[0]
print country
country_list.append(country)
#print "finished " + urlx + ", added" + country
country_list
#print 'finished making country_list'

England


['England']

In [59]:
#this generates a list of countries, the list is in the order of the urls that we'd generated of the grounds
#the fact that they're in the same order will prove useful later, as we'll be merging this list with that file
#or 'concatenating' which is the technical term used to describe the process


import time
from lxml import html
from lxml.cssselect import CSSSelector
from random import randint

country_list = []
#d = 0
for urlx in new_list:
    print "processing " + urlx
    responseq = session_requests.get(urlx)
    #d += 1
    #zrs = str(d).zfill(3)
    #number = zrs + '.html'
    #filenamey = 'grounds/' + number
    #fnamez = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenamey)
    #with open(fnamez, "w") as filex:
     #   filex.write(responseq.content)
    #  filex.close()
    #print "written " + urlx
    #with open(fnamez) as filez:
    #    sourcex = filez.read()
    #treex = html.document_fromstring(sourcex)
    #treex = html.document_fromstring(responseq.text)
    #treex = html.document_fromstring(responseq.content)
    treex = html.document_fromstring(responseq.text)
    #country = treex.xpath('//*[@id="columnLeft"]/table/tbody/tr[3]/td[2]/text()')[0]
    country = treex.xpath('//b[text()="Country:"]/following::td[1]/text()')[0]
    print country
    country_list.append(country)
    print "finished " + urlx + ", added " + country
    delay = randint(10,15)
    time.sleep(delay)

print 'finished making country_list'

processing http://cricketarchive.com/Archive/Grounds/11/321.html
England
finished http://cricketarchive.com/Archive/Grounds/11/321.html, added England
processing http://cricketarchive.com/Archive/Grounds/11/596.html
England
finished http://cricketarchive.com/Archive/Grounds/11/596.html, added England
processing http://cricketarchive.com/Archive/Grounds/11/618.html
England
finished http://cricketarchive.com/Archive/Grounds/11/618.html, added England
.
.
.
processing http://cricketarchive.com/Archive/Grounds/60/1297.html
Ireland
finished http://cricketarchive.com/Archive/Grounds/60/1297.html, added Ireland
processing http://cricketarchive.com/Archive/Grounds/2/83.html
Australia
finished http://cricketarchive.com/Archive/Grounds/2/83.html, added Australia
finished making country_list


In [60]:
print country_list

['England', 'England', 'England', 'England', 'Sri Lanka', 'England', 'Sri Lanka', 'England', 'England', 'Sri Lanka', 'Pakistan', 'Pakistan', 'Pakistan', 'Australia', 'India', 'Australia', 'Australia', 'Australia', 'New Zealand', 'Australia', 'New Zealand', 'New Zealand', 'West Indies', 'West Indies', 'West Indies', 'West Indies', 'West Indies', 'Pakistan', 'Pakistan', 'New Zealand', 'Zimbabwe', 'Zimbabwe', 'South Africa', 'South Africa', 'Sri Lanka', 'South Africa', 'South Africa', 'India', 'India', 'India', 'India', 'Australia', 'Pakistan', 'Pakistan', 'India', 'India', 'India', 'Sri Lanka', 'Zimbabwe', 'India', 'India', 'New Zealand', 'New Zealand', 'Pakistan', 'India', 'South Africa', 'Pakistan', 'India', 'West Indies', 'Bangladesh', 'Sri Lanka', 'South Africa', 'Pakistan', 'Bangladesh', 'United Arab Emirates', 'South Africa', 'South Africa', 'West Indies', 'West Indies', 'Australia', 'Australia', 'Bangladesh', 'Bangladesh', 'Bangladesh', 'West Indies', 'New Zealand', 'Bangladesh', 

In [80]:
len(new_list)

189

In [81]:
len(country_list)

189

In [82]:
len(new_list2)

189

In [83]:
new_list == new_list2

False

In [84]:
new_list

['http://cricketarchive.com/Archive/Grounds/11/321.html',
 'http://cricketarchive.com/Archive/Grounds/11/596.html',
 'http://cricketarchive.com/Archive/Grounds/11/618.html',
.
.
.
 'http://cricketarchive.com/Archive/Grounds/11/570.html',
 'http://cricketarchive.com/Archive/Grounds/19/1344.html',
 'http://cricketarchive.com/Archive/Grounds/60/1297.html',
 'http://cricketarchive.com/Archive/Grounds/2/83.html']

In [85]:
new_list2

['/Archive/Grounds/11/321.html',
 '/Archive/Grounds/11/596.html',
 '/Archive/Grounds/11/618.html',
.
.
.
 '/Archive/Grounds/11/570.html',
 '/Archive/Grounds/25/1707.html',
 '/Archive/Grounds/19/1344.html',
 '/Archive/Grounds/60/1297.html',
 '/Archive/Grounds/2/83.html']

In [87]:
#making a dataframe
import pandas as pd

combined = [('ground_url', new_list2),('ground_country', country_list)]
df_new = pd.DataFrame.from_items(combined)
df_new.head(n=5)


,ground_url,ground_country
0,/Archive/Grounds/11/321.html,England
1,/Archive/Grounds/11/596.html,England
2,/Archive/Grounds/11/618.html,England
3,/Archive/Grounds/11/570.html,England
4,/Archive/Grounds/25/1707.html,Sri Lanka


In [88]:
print df_new.shape

(189, 2)


In [89]:
df = pd.read_csv('ground_urls.csv')
df.head(n=5)

,ground,ground_url
0,"Edgbaston, Birmingham",/Archive/Grounds/11/321.html
1,"Lord's Cricket Ground, St John's Wood",/Archive/Grounds/11/596.html
2,"Old Trafford, Manchester",/Archive/Grounds/11/618.html
3,"Headingley, Leeds",/Archive/Grounds/11/570.html
4,"R Premadasa Stadium, Colombo",/Archive/Grounds/25/1707.html


In [90]:
print df.shape

(5087, 2)


In [91]:
#if I remember correctly this 'merging' actually does a kind of a vlookup thing if you're familiar with that function
#from excel, so a new column 'ground_country' is created containing the right country for each ground

df3 = df.merge(df_new, on='ground_url', how='left')
df3.head(n=5)

,ground,ground_url,ground_country
0,"Edgbaston, Birmingham",/Archive/Grounds/11/321.html,England
1,"Lord's Cricket Ground, St John's Wood",/Archive/Grounds/11/596.html,England
2,"Old Trafford, Manchester",/Archive/Grounds/11/618.html,England
3,"Headingley, Leeds",/Archive/Grounds/11/570.html,England
4,"R Premadasa Stadium, Colombo",/Archive/Grounds/25/1707.html,Sri Lanka


In [99]:
print df3.shape

(5087, 3)


In [121]:
df4 = df3.drop('ground',1)
df4.head(n=5)

,ground_url,ground_country
0,/Archive/Grounds/11/321.html,England
1,/Archive/Grounds/11/596.html,England
2,/Archive/Grounds/11/618.html,England
3,/Archive/Grounds/11/570.html,England
4,/Archive/Grounds/25/1707.html,Sri Lanka


In [122]:
df4.shape

(5087, 2)

In [123]:
df5 = pd.read_csv('url_store.csv')
df5.head(n=5)

,format,number,date,year,team1,team2,ground,url
0,test,test_0001,05 Jun 1997,1997,England,Australia,"Edgbaston, Birmingham",/Archive/Scorecards/64/64098.html
1,test,test_0002,19 Jun 1997,1997,England,Australia,"Lord's Cricket Ground, St John's Wood",/Archive/Scorecards/64/64171.html
2,test,test_0003,03 Jul 1997,1997,England,Australia,"Old Trafford, Manchester",/Archive/Scorecards/64/64259.html
3,test,test_0004,24 Jul 1997,1997,England,Australia,"Headingley, Leeds",/Archive/Scorecards/64/64378.html
4,test,test_0005,02 Aug 1997,1997,Sri Lanka,India,"R Premadasa Stadium, Colombo",/Archive/Scorecards/64/64422.html


In [124]:
print df5.shape

(5087, 8)


In [126]:
#the dataframe below is the result of 'concatenating' which just means joining two things along a side or 'axis'
#this dataframe will eventually become a csv that we use to extract the country that each ground is based in
#dont think there was any other use of this file to be honest! everything else in the file could be extracted from the 
#scraping we're doing next

df6 = pd.concat([df5, df4], axis=1)
df6.head(n=5)

,format,number,date,year,team1,team2,ground,url,ground_url,ground_country
0,test,test_0001,05 Jun 1997,1997,England,Australia,"Edgbaston, Birmingham",/Archive/Scorecards/64/64098.html,/Archive/Grounds/11/321.html,England
1,test,test_0002,19 Jun 1997,1997,England,Australia,"Lord's Cricket Ground, St John's Wood",/Archive/Scorecards/64/64171.html,/Archive/Grounds/11/596.html,England
2,test,test_0003,03 Jul 1997,1997,England,Australia,"Old Trafford, Manchester",/Archive/Scorecards/64/64259.html,/Archive/Grounds/11/618.html,England
3,test,test_0004,24 Jul 1997,1997,England,Australia,"Headingley, Leeds",/Archive/Scorecards/64/64378.html,/Archive/Grounds/11/570.html,England
4,test,test_0005,02 Aug 1997,1997,Sri Lanka,India,"R Premadasa Stadium, Colombo",/Archive/Scorecards/64/64422.html,/Archive/Grounds/25/1707.html,Sri Lanka


In [127]:
df6.shape

(5087, 10)

In [128]:
df6.to_csv('url_store_complete.csv')

In [129]:
df6.shape

(5087, 10)

In [20]:
import csv
reader = csv.DictReader(open('url_store_complete.csv'))

gdict = {}
for row in reader:
    key = row.pop('number')
    gdict[key] = row
#print gdict

In [5]:
#just testing out some code that will be part of some other code later on
#if you're someone who uses python to scrape websites, look at the comment below, it'll help avoid a lot of grief


from lxml import html
import os

filenamey = 'code_check/odi_1915.html'
fnamez =  os.path.join(os.path.dirname(os.path.realpath('__file__')), filenamey)
with open(fnamez) as filez:
    sourcex = filez.read()
treex = html.document_fromstring(sourcex)

#print(html.tostring(treex, pretty_print=True))

#was stuck on this for a day, seems that the xpath returned by firebug or chrome's inspector tools won't work
#sometimes in python. According to this page http://stackoverflow.com/questions/32015083/python-xpath-returns-an-empty-list
#the way your browser parses html and lxml parses html is different
#for example, the xpath I used first to get a certain element was this:
#'//*[@id="columnLeft"]/table[1]/tbody/tr[1]/td[2]/b/font/center/a[1]/text()' . This was returned by Chrome but didn't work
#here because python parsed the page differently. The xpath returned by the browser tools also introduced a tbody tag
#which wasn't there in the original html. Ultimately used the code below.

divx = treex.xpath("//*[@id='columnLeft']/table[1]/tr[1]/td[2]/center/a[1]/text()")[0]

print divx

Ireland


In [21]:
#this is the most important piece of code in this whole ipython notebook
#what this does is go through the scoreboard for each t20, ODI and test match from the past 25 years and gets all the
#information we will use later on. Everything from the umpire's name and who won the toss to traditional scoreboard data
#such as the number of balls each batsman faced and the number of maidens overs bowled

#at the end of this code block, three json files will be created, one each for the data on tests, ODIs and T20s
#don't know if anyone non-techie is still looking at this, but json is a better format than csv when some 
#scoreboards contain more data than others, it accommodates varying levels of depth, if that makes sense



from collections import defaultdict
#import collections
import json
import os
from lxml import html
import re
import io

formats = ["test","odi","t20"]
for formatx in formats:
    
    tree = lambda: defaultdict(tree)
    jdict = tree()  #this ensures a fresh dictionary/json is created for each cricket format
    #so the code above has been used to create a dynamic tree.Got the code from http://sopython.com/wiki/Fancy_Python_Tricks and 
    #https://gist.github.com/hrldcpr/2012250 , seems it's something called autovivification. Whatever.
    
    folderx = formatx + '_cards'
    for i in os.listdir(folderx):
        if i not in ['odi_2018.html']: #this is a troublesome file/scoreboard that I excluded
            #loops through all the files in the folder
            print "processing " + i
            filex = formatx + '_cards/' + i
            filey =  os.path.join(os.path.dirname(os.path.realpath('__file__')), filex)
            with open(filey) as filez:
                sourcex = filez.read()
                treex = html.document_fromstring(sourcex)
                #print(html.tostring(treex, pretty_print=True))
                jdict[i]['format'] = (re.findall(r"(.*)_.*", i))[0] #whether test, odi, t20
                jdict[i]['team1'] = treex.xpath('//*[@id="columnLeft"]/table[1]/tr[1]/td[2]/center/a[1]/text()')[0]
                jdict[i]['team2'] = treex.xpath('//*[@id="columnLeft"]/table[1]/tr[1]/td[2]/center/a[2]/text()')[0]
                jdict[i]['tour'] = treex.xpath('//*[@id="columnLeft"]/table[1]/tr[2]/td[2]/center/a/text()')[0]

                jdict[i]['venue_text']= ''.join(treex.xpath('//*[@id="columnLeft"]/table[1]//tr//td[contains(text(),"Venue")]\
                /following-sibling::td[1]//text()'))
                #print jdict[i]['venue_text']
                #you can find out if a match is day-night or not by doing a search for that substring against venue_text

                jdict[i]['year']= (re.findall(r'\d{4}',jdict[i]['venue_text']))[0]
                jdict[i]['venue']= treex.xpath('//*[@id="columnLeft"]/table[1]//tr//td[contains(text(),"Venue")]/\
                                    following-sibling::td[1]/a/text()')[0]

                #details below are from gdict, which is url_store_complete.csv in a dictionary, gets us countries of grounds
                j = (re.findall(r'(.*).html', i))[0]
                jdict[i]['start_date']= gdict[j]['date']
                jdict[i]['venue_country']= gdict[j]['ground_country']
                jdict[i]['venue_url']= gdict[j]['ground_url']
                jdict[i]['scoreboard_url']= gdict[j]['url']
                try:
                    jdict[i]['toss_text']= treex.xpath('//*[@id="columnLeft"]/table[1]//tr//td[contains(text(),"Toss")]\
                    /following-sibling::td[1]/text()')[0]
                    jdict[i]['toss_winner']= (re.findall(r'(.*) won .*',jdict[i]['toss_text']))[0]
                except:
                    pass
                else:
                    jdict[i]['toss_winner_choice']= (re.findall(r'.* decided to (.*)',jdict[i]['toss_text']))[0]
                jdict[i]['result_text']= ''.join(treex.xpath('//*[@id="columnLeft"]/table[1]//tr//td[contains(text(),"Result")]\
                /following-sibling::td[1]//text()'))
                #print jdict[i]['result_text']
                if ' won by ' in jdict[i]['result_text']:
                    jdict[i]['winner']= (re.findall(r'(.*) won by .*',jdict[i]['result_text']))[0]
                    jdict[i]['winning_margin']= (re.findall(r'.* won by (.*)',jdict[i]['result_text']))[0]

                    #if you want to find out draws in tests, just search for draw against result_text
                    #do a word cloud of result text to see what other outcomes are possible - cancelled, tied, suspended etc.
                    #also if you're checking for how many matches ended in actual wins, search for " won by ", not the word
                    #'won', because that might bring in results for t20 matches decided in super-overs, where there's no winning margin
                try:
                    jdict[i]['umpires_text'] = ''.join(treex.xpath('//*[@id="columnLeft"]/table[1]//tr//td[contains(text(),"Umpires")]\
                    /following-sibling::td[1]//text()'))
                    #print jdict[i]['umpires_text']
                except:
                    pass
                else:
                    jdict[i]['umpires_names'] = treex.xpath('//*[@id="columnLeft"]/table[1]//tr//td[contains(text(),"Umpires")]\
                    /following-sibling::td[1]//a/text()')
                    jdict[i]['umpires_urls'] = treex.xpath('//*[@id="columnLeft"]/table[1]//tr//td[contains(text(),"Umpires")]\
                    /following-sibling::td[1]//a/@href')
                try:
                    jdict[i]['man_match']= treex.xpath('//*[@id="columnLeft"]/table[1]//tr//td[contains(text(),"Man of the Match")]\
                    /following-sibling::td[1]/a/text()')[0]
                    jdict[i]['man_match_url']= treex.xpath('//*[@id="columnLeft"]/table[1]//tr//td[contains(text(),"Man of the Match")]\
                    /following-sibling::td[1]/a/@href')[0]
                except:
                    pass

                #now to get the batting and bowling stats

                innings = treex.xpath('//b[text()[contains(.,"innings") and not(contains(.,"super"))]]')
                #got the code above from http://stackoverflow.com/questions/3655549
                #/xpath-containstext-some-string-doesnt-work-when-used-with-node-with-more
                #added the not contains 'super' to exclude super-over innings from t20 matches
                #print innings (this will give a list of <b> or bold nodes)

                order =0

                for inning in innings:
                    #inning_name = ''.join([inning.xpath('a/text()')[0],inning.xpath('text()')[0]]) this didn't work
                    #inning_name = inning.text_content().lstrip(' ')
                    order += 1
                    inning_order = order

                    #print inning_name
                    #print type(inning_name)
                    team_batting = inning.xpath('a/text()')[0]
                    #print team_batting

                    #CHECKS
                    if inning.xpath('following::td[1]/b/text()')[0] != 'Runs':
                        print "Runs figures wrong"
                    if inning.xpath('following::td[2]/b/text()')[0] != 'Balls':
                        print "Balls figures wrong"
                    if inning.xpath('following::td[4]/b/text()')[0] != '4s':
                        print "4s figures wrong"
                    if inning.xpath('following::td[5]/b/text()')[0] != '6s':
                        print "6s figures wrong"


                    batsman_list = inning.xpath('following::tr[position() >= 1 and not(position() > 11)]/td[1]')
                    #print batsman_list
                    d = 0
                    for batsman in batsman_list:
                        d += 1
                        batsman_name = batsman.xpath('a/text()')[0]
                        #print batsman_name
                        batsman_order = d

                        jdict[i]['innings'][inning_order]['team_batting'][team_batting]['batsman_all'][batsman_order]\
                            [batsman_name]['batsman_url'] = batsman.xpath('a/@href')[0]

                        jdict[i]['innings'][inning_order]['team_batting'][team_batting]['batsman_all'][batsman_order]\
                            [batsman_name]['batsman_dismissal_text'] = ''.join(batsman.xpath('following::td[1]//text()'))
                        jdict[i]['innings'][inning_order]['team_batting'][team_batting]['batsman_all'][batsman_order]\
                            [batsman_name]['batsman_dismissal_unlinked'] = ' '.join(batsman.xpath('following::td[1]/text()'))
                        jdict[i]['innings'][inning_order]['team_batting'][team_batting]['batsman_all'][batsman_order]\
                            [batsman_name]['batsman_dismissal_names'] = batsman.xpath('following::td[1]//a/text()')
                        jdict[i]['innings'][inning_order]['team_batting'][team_batting]['batsman_all'][batsman_order]\
                            [batsman_name]['batsman_dismissal_urls'] = batsman.xpath('following::td[1]//a/@href') 

                        if 'absent hurt' in ''.join(batsman.xpath('following::td[1]//text()')):
                            pass
                        else:
                            jdict[i]['innings'][inning_order]['team_batting'][team_batting]['batsman_all'][batsman_order]\
                                [batsman_name]['batsman_runs'] = batsman.xpath('following::td[2]/text()')[0]
                            jdict[i]['innings'][inning_order]['team_batting'][team_batting]['batsman_all'][batsman_order]\
                                [batsman_name]['batsman_balls'] = batsman.xpath('following::td[3]/text()')[0]
                            jdict[i]['innings'][inning_order]['team_batting'][team_batting]['batsman_all'][batsman_order]\
                                [batsman_name]['batsman_4s'] = batsman.xpath('following::td[5]/text()')[0]
                            jdict[i]['innings'][inning_order]['team_batting'][team_batting]['batsman_all'][batsman_order]\
                                [batsman_name]['batsman_6s'] = batsman.xpath('following::td[6]/text()')[0]


                        #if you want to find out if someone is not out, just search for substring 'not out' against dismissal_unlinked
                        #if you want to find if someone is stumped, search for 'st' etc.
                        #do a word cloud to find out various methods of dismissal
                        #word cloud will also have "did not bat" "retired hurt" etc.
                        #in dismissal text, see if number of run outs are rising, is that a sign of poor batsmanship?
                        #of how much more frenzied things are getting in forms of cricket other than t20?
                        #just do a search for 'run out' substring in batsman_dismissal_text


                    try:
                        jdict[i]['innings'][inning_order]['team_batting'][team_batting]['extras']['text'] = \
                                inning.xpath('following::tr[12]/td[2]/text()')[0]
                    except:
                        pass
                    else:
                        #if int(inning.xpath('following::tr[12]/td[3]/text()')[0]) == 0:
                        jdict[i]['innings'][inning_order]['team_batting'][team_batting]['extras']['runs'] = \
                                inning.xpath('following::tr[12]/td[3]/text()')[0]
                        extras_types = \
                             re.findall(r"[a-z]+", jdict[i]['innings'][inning_order]['team_batting'][team_batting]['extras']['text'])
                        extras_figures = \
                             re.findall(r"[0-9]+", jdict[i]['innings'][inning_order]['team_batting'][team_batting]['extras']['text'])
                        extras_dict = dict(zip(extras_types, extras_figures))
                        jdict[i]['innings'][inning_order]['team_batting'][team_batting]['extras']['details'] = extras_dict

                    #the details above give you number of wides, legbyes, byes etc.


                    try:
                        jdict[i]['innings'][inning_order]['team_batting'][team_batting]['total']['text'] =\
                        inning.xpath('following::tr[13]/td[2]/text()')[0]
                        jdict[i]['innings'][inning_order]['team_batting'][team_batting]['total']['overs'] = \
                            (re.findall(r".*, (.*) overs\)", jdict[i]['innings'][inning_order]['team_batting'][team_batting]['total']['text']))[0]
                    except:
                        pass
                    else:
                        jdict[i]['innings'][inning_order]['team_batting'][team_batting]['total']['runs'] =\
                            inning.xpath('following::tr[13]/td[3]/text()')[0]
                        if 'all out' in jdict[i]['innings'][inning_order]['team_batting'][team_batting]['total']['text']:
                            jdict[i]['innings'][inning_order]['team_batting'][team_batting]['total']['wickets'] = '10'
                                                                                        #dont put 'all out', keep all column
                                                                                           #values as single type
                        if 'wickets' in jdict[i]['innings'][inning_order]['team_batting'][team_batting]['total']['text']:
                            jdict[i]['innings'][inning_order]['team_batting'][team_batting]['total']['wickets'] =\
                            (re.findall(r"\((.*) wickets, .*", jdict[i]['innings'][inning_order]['team_batting'][team_batting]['total']['text']))[0]
                        if '1 wicket, ' in jdict[i]['innings'][inning_order]['team_batting'][team_batting]['total']['text']:
                            jdict[i]['innings'][inning_order]['team_batting'][team_batting]['total']['wickets'] = '1'
                        
                        if 'no wicket'in jdict[i]['innings'][inning_order]['team_batting'][team_batting]['total']['text']:
                            jdict[i]['innings'][inning_order]['team_batting'][team_batting]['total']['wickets'] = '0'

                    #note that sometimes teams declare too, you can check for that against the total text

                    batsman_dismissed_names = inning.xpath('following::tr[15]/td[1]//a/text()')
                    batsman_dismissed_urls = inning.xpath('following::tr[15]/td[1]//@href')
                    #keep it as batsman throughout, don't make it batsmen just so it's gramatically correct
                    #dont trip by using batsman in some places and batsmen in other places

                    try:
                        fall_text = ''.join(inning.xpath('following::tr[15]/td[1]//text()'))
                        jdict[i]['innings'][inning_order]['team_batting'][team_batting]['fall_wickets']['text'] = fall_text
                        fall_list = fall_text.split("), ")
                        throwaway = (re.findall(r"([0-9]+)-.*", fall_list[0]))[0]
                    except:
                        pass
                    else:
                        for fall in fall_list:
                            f = (re.findall(r"([0-9]+)-.*", fall))[0]
                            #print f
                            jdict[i]['innings'][inning_order]['team_batting'][team_batting]['fall_wickets'][f]['score']=\
                                (re.findall(r".*-(.*) \(.*", fall))[0]
                            try:
                                jdict[i]['innings'][inning_order]['team_batting'][team_batting]['fall_wickets'][f]['over']=\
                                (re.findall(r".*, (.*) o.*", fall))[0]
                            except:
                                pass
                            #jdict[i][inning_name][team_batting]['fall_wickets'][f]['name']=(re.findall(r".* \((.*), .*", fall))[0]
                            jdict[i]['innings'][inning_order]['team_batting'][team_batting]['fall_wickets'][f]['name']=\
                                batsman_dismissed_names[int(f)-1]
                            jdict[i]['innings'][inning_order]['team_batting'][team_batting]['fall_wickets'][f]['url']=\
                                batsman_dismissed_urls[int(f)-1]

                    #bowling figures

                    try:
                        team_bowling = inning.xpath('following::table[1]/tr[1]/td[1]/b/a/text()')[0]
                    except:
                        pass
                    else:
                        #CHECK if column labels are where we expect them to be
                        if inning.xpath('following::table[1]/tr[1]/td[2]/b/text()')[0] != 'Overs':
                            print "Overs figures wrong"
                        if inning.xpath('following::table[1]/tr[1]/td[3]/b/text()')[0] != 'Mdns':
                            print "Maidens figures wrong"
                        if inning.xpath('following::table[1]/tr[1]/td[4]/b/text()')[0] != 'Runs':
                            print "Runs figures wrong"
                        if inning.xpath('following::table[1]/tr[1]/td[5]/b/text()')[0] != 'Wkts':
                            print "Wickets figures wrong"

                        bowler_rows = inning.xpath('following::table[1]//tr')
                        #print bowler_rows

                        for row in bowler_rows[1:-1]:
                            bowler_name = row.xpath('td[1]/a/text()')[0]
                            jdict[i]['innings'][inning_order]['team_bowling'][team_bowling]['bowlers_all'][bowler_name]['bowler_url']=\
                                row.xpath('td[1]/a/@href')[0]
                            overs = row.xpath('td[2]/text()')[0]
                            jdict[i]['innings'][inning_order]['team_bowling'][team_bowling]['bowlers_all'][bowler_name]['overs']= \
                                overs.replace("&nbsp;",'')
                            jdict[i]['innings'][inning_order]['team_bowling'][team_bowling]['bowlers_all'][bowler_name]['maidens']=\
                                row.xpath('td[3]/text()')[0]
                            jdict[i]['innings'][inning_order]['team_bowling'][team_bowling]['bowlers_all'][bowler_name]['runs']=\
                                row.xpath('td[4]/text()')[0]
                            jdict[i]['innings'][inning_order]['team_bowling'][team_bowling]['bowlers_all'][bowler_name]['wickets']=\
                                row.xpath('td[5]/text()')[0]

            print "finished " + i + '\n'

    #code below taken from http://stackoverflow.com/questions/2594810/removing-non-breaking-spaces-from-strings-using-python
    #and http://stackoverflow.com/questions/12309269/how-do-i-write-json-data-to-a-file-in-python
    
    jstring = json.dumps(jdict, indent=4, sort_keys=True)
    jstring2 = jstring.replace("\u00a0","")

    filenameg = formatx + '_data.json' 
    fnameg = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenameg)

    with io.open(fnameg, 'w') as json_file:
        json_file.write(unicode(jstring2))

print "Processing done"

processing test_0001.html
finished test_0001.html

processing test_0002.html
finished test_0002.html

.
.
.
processing t20_0616.html
finished t20_0616.html

processing t20_0617.html
finished t20_0617.html

Processing done


In [107]:
#am loading the different jsons that have been created into different dataframes just to do some preliminary 
#data checks and see if everything is kosher

#import pandas as pd
test_df = pd.read_json('test_data.json')
odi_df = pd.read_json('odi_data.json')
t20_df = pd.read_json('t20_data.json')

#import json

#with open('test_data.json', 'r') as f:
#     test_df = json.load(f)

#with open('odi_data.json', 'r') as f:
#     odi_df = json.load(f)

#with open('t20_data.json', 'r') as f:
#     t20_df = json.load(f)

nation_list = ['Australia','England','New Zealand','West Indies','India','Sri Lanka','Pakistan','Bangladesh',\
               'South Africa','Zimbabwe']

In [2]:
#one of the checks i did, essentially here i'm asking the code to add the runs that each batsman has scored in an innings
#, add it to the extras and see if it equals the total score that we'd extracted separately

for formatx in [test_df, odi_df, t20_df]:
    for page in formatx.keys():
        #print page
        try:
            formatx[page]['innings'].keys()
        except:
            continue
        else:
            for inning in formatx[page]['innings'].keys():
                for team in formatx[page]['innings'][inning]['team_batting'].keys():
                    non_extras = 0
                    try:
                        extras = int(formatx[page]['innings'][inning]['team_batting'][team]['extras']['runs'])
                    except:
                        extras = 0
                    try:
                        total = int(formatx[page]['innings'][inning]['team_batting'][team]['total']['runs'])
                    except:
                        print "No total in " + page
                    for position in formatx[page]['innings'][inning]['team_batting'][team]['batsman_all'].keys():
                        for name in formatx[page]['innings'][inning]['team_batting'][team]['batsman_all'][position].keys():
                            try:
                                non_extras = non_extras + int(formatx[page]['innings'][inning]['team_batting'][team]\
                                                              ['batsman_all'][position][name]['batsman_runs'])
                            except:
                                non_extras = non_extras + 0
                    if non_extras + extras != total:
                        print "Check " + page + " and team " + team + " and inning no. " + inning

No total in test_0046.html
Check test_0046.html and team Sri Lanka and inning no. 2
No total in test_0124.html
Check test_0124.html and team India and inning no. 2
.
.
.
No total in t20_0538.html
Check t20_0538.html and team Scotland and inning no. 1
No total in t20_0538.html
Check t20_0538.html and team Ireland and inning no. 2


In [12]:
#this is just another check of the data we've scraped
#this code was done to find out matches where there results other than a victory or a draw, such as match tied, cancelled
#suspended etc. see if there are any troublesome matches with weird scoreboards that might have produced
#weird results even if no errors were put out during the processing


#so it seems the text given out by doing [result_text] is unicode, doing 'if any in for that' as I've done below
#is the easiest way to search for a substring in the text

listx = ['won by','drawn']

for formatx in [test_df, odi_df, t20_df]:
    for page in formatx.keys():
        if any(sub in formatx[page]['result_text'] for sub in listx):
            continue
        else:
            print 'check ' + page

check test_0170.html
check test_0292.html
check test_0293.html
.
.
.
check t20_0566.html
check t20_0612.html
check t20_0614.html


In [64]:
#this block is to get the totals distributed by year
#ie. how many totals between 0-200, how many between 201-400 etc.
#note that this only considers those totals from ODIs and T20s where the team batted first and those totals from tests
#made in a team's first innings

#the reason why I'm only considering first innings totals is that first innings totals typically represent a team
#trying to score as much as it can. The second innings totals are typically about setting a good enough or 
#safe enough target to set the other team. So if a team is set only 100 to win a test, this code doesn't penalise
#them for only scoring that much. That was the target they were set, they could have scored more, but didn't need to.
#Same is the case for T20 and ODI matches, only the totals of the teams batting first were recorded.

from collections import defaultdict
import json
import os
import io

test_df.name = 'test_df'
odi_df.name = 'odi_df'
t20_df.name = 't20_df'

for formatx in [test_df, odi_df, t20_df]:

    jdict = defaultdict(lambda: defaultdict(lambda: defaultdict(int))) 

    for page in formatx.keys():
        year = formatx[page]['year']
        try:
            formatx[page]['innings'].keys()
        except:
            continue
        else:
            for inning in formatx[page]['innings'].keys():
                if formatx.name == test_df.name:
                    if int(inning) == 1 or int(inning) == 2:
                        for team in formatx[page]['innings'][inning]['team_batting'].keys():
                            if team in nation_list:
                                try:
                                    int(formatx[page]['innings'][inning]['team_batting'][team]['total']['runs'])
                                except:
                                    continue
                                else:
                                    if int(formatx[page]['innings'][inning]['team_batting'][team]['total']['runs']) > 600:
                                        jdict[team][year]['600 plus'] += 1
                                    elif int(formatx[page]['innings'][inning]['team_batting'][team]['total']['runs']) > 400:
                                        jdict[team][year]['401-600'] += 1
                                    elif int(formatx[page]['innings'][inning]['team_batting'][team]['total']['runs']) > 200:
                                        jdict[team][year]['201-400'] += 1
                                    else:
                                        jdict[team][year]['0-200'] += 1 #one
                    else:
                        continue
                elif formatx.name == odi_df.name:
                    if int(inning) == 1 or int(inning) == 2:
                        for team in formatx[page]['innings'][inning]['team_batting'].keys():
                            if team in nation_list:
                                try:
                                    int(formatx[page]['innings'][inning]['team_batting'][team]['total']['runs'])
                                except:
                                    continue
                                else:
                                    if int(formatx[page]['innings'][inning]['team_batting'][team]['total']['runs']) > 300:
                                        jdict[team][year]['300 plus'] += 1
                                    elif int(formatx[page]['innings'][inning]['team_batting'][team]['total']['runs']) > 200:
                                        jdict[team][year]['201-300'] += 1
                                    elif int(formatx[page]['innings'][inning]['team_batting'][team]['total']['runs']) > 100:
                                        jdict[team][year]['101-200'] += 1
                                    else:
                                        jdict[team][year]['0-100'] += 1 #two
                    else:
                        continue
                else:
                    if int(inning) == 1:
                        for team in formatx[page]['innings'][inning]['team_batting'].keys():
                            if team in nation_list:
                                try:
                                    int(formatx[page]['innings'][inning]['team_batting'][team]['total']['runs'])
                                except:
                                    continue
                                else:
                                    if int(formatx[page]['innings'][inning]['team_batting'][team]['total']['runs']) > 200:
                                        jdict[team][year]['200 plus'] += 1
                                    elif int(formatx[page]['innings'][inning]['team_batting'][team]['total']['runs']) > 100:
                                        jdict[team][year]['101-200'] += 1
                                    else:
                                        jdict[team][year]['0-100'] += 1 #three
                    else:
                        continue

    jstring = json.dumps(jdict, indent=4, sort_keys=True)
    
    filenameg = formatx.name + '_totals_by_year.json' 
    fnameg = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenameg)

    with io.open(fnameg, 'w') as json_file:
        json_file.write(unicode(jstring))

print "Processing done"

Processing done


In [ ]:
#this block is to get the individual scores distributed by year and team
#individual scores made in all innings considered for this, no distinction between first innings and second innings
#as in the code block above

from collections import defaultdict
import json
import os
import io

test_df.name = 'test_df'
odi_df.name = 'odi_df'
t20_df.name = 't20_df'

for formatx in [test_df, odi_df, t20_df]:

    jdict = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
    
    for page in formatx.keys():
        print page
        year = formatx[page]['year']
        try:
            formatx[page]['innings'].keys()
        except:
            continue
        else:
            for inning in formatx[page]['innings'].keys():
                if formatx.name == test_df.name:
                    for team in formatx[page]['innings'][inning]['team_batting'].keys():
                        if team in nation_list:
                            try:
                                int(formatx[page]['innings'][inning]['team_batting'][team]['total']['runs'])
                            except:
                                continue
                            else:
                                for order in formatx[page]['innings'][inning]['team_batting'][team]['batsman_all'].keys():
                                    for namex in formatx[page]['innings'][inning]['team_batting'][team]['batsman_all'][order].keys():
                                        try:
                                            int(formatx[page]['innings'][inning]['team_batting'][team]['batsman_all'][order]\
                                               [namex]['batsman_runs'])
                                        except:
                                            continue
                                        else:
                                            if int(formatx[page]['innings'][inning]['team_batting'][team]['batsman_all'][order]\
                                                   [namex]['batsman_runs']) > 299:
                                                jdict[team][year]['300 & over'] += 1
                                            elif int(formatx[page]['innings'][inning]['team_batting'][team]['batsman_all'][order]\
                                                   [namex]['batsman_runs']) > 199:
                                                jdict[team][year]['200-299'] += 1
                                            elif int(formatx[page]['innings'][inning]['team_batting'][team]['batsman_all'][order]\
                                                   [namex]['batsman_runs']) > 99:
                                                jdict[team][year]['100-199'] += 1
                                            else:
                                                jdict[team][year]['0-99'] += 1
                                                                        
                elif formatx.name == odi_df.name:
                    for team in formatx[page]['innings'][inning]['team_batting'].keys():
                        if team in nation_list:
                            try:
                                int(formatx[page]['innings'][inning]['team_batting'][team]['total']['runs'])
                            except:
                                continue
                            else:
                                for order in formatx[page]['innings'][inning]['team_batting'][team]['batsman_all'].keys():
                                    for namex in formatx[page]['innings'][inning]['team_batting'][team]['batsman_all'][order].keys():
                                        try:
                                            int(formatx[page]['innings'][inning]['team_batting'][team]['batsman_all'][order]\
                                               [namex]['batsman_runs'])
                                        except:
                                            continue
                                        else:
                                            if int(formatx[page]['innings'][inning]['team_batting'][team]['batsman_all'][order]\
                                                   [namex]['batsman_runs']) > 149:
                                                jdict[team][year]['150 & over'] += 1
                                            elif int(formatx[page]['innings'][inning]['team_batting'][team]['batsman_all'][order]\
                                                   [namex]['batsman_runs']) > 99:
                                                jdict[team][year]['100-149'] += 1
                                            elif int(formatx[page]['innings'][inning]['team_batting'][team]['batsman_all'][order]\
                                                   [namex]['batsman_runs']) > 49:
                                                jdict[team][year]['50-99'] += 1
                                            else:
                                                jdict[team][year]['0-49'] += 1
                else:
                    for team in formatx[page]['innings'][inning]['team_batting'].keys():
                        if team in nation_list:
                            try:
                                int(formatx[page]['innings'][inning]['team_batting'][team]['total']['runs'])
                            except:
                                continue
                            else:
                                for order in formatx[page]['innings'][inning]['team_batting'][team]['batsman_all'].keys():
                                    for namex in formatx[page]['innings'][inning]['team_batting'][team]['batsman_all'][order].keys():
                                        try:
                                            int(formatx[page]['innings'][inning]['team_batting'][team]['batsman_all'][order]\
                                               [namex]['batsman_runs'])
                                        except:
                                            continue
                                        else:
                                            if int(formatx[page]['innings'][inning]['team_batting'][team]['batsman_all'][order]\
                                                   [namex]['batsman_runs']) > 149:
                                                jdict[team][year]['150 & over'] += 1
                                            elif int(formatx[page]['innings'][inning]['team_batting'][team]['batsman_all'][order]\
                                                   [namex]['batsman_runs']) > 99:
                                                jdict[team][year]['100-149'] += 1
                                            elif int(formatx[page]['innings'][inning]['team_batting'][team]['batsman_all'][order]\
                                                   [namex]['batsman_runs']) > 49:
                                                jdict[team][year]['50-99'] += 1
                                            else:
                                                jdict[team][year]['0-49'] += 1
                                        
    jstring = json.dumps(jdict, indent=4, sort_keys=True)
    
    filenameg = formatx.name + '_indiv_scores_by_year.json' 
    fnameg = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenameg)

    with io.open(fnameg, 'w') as json_file:
        json_file.write(unicode(jstring))


print "Processing done"

In [ ]:
#this block is to get the runs made and the number of overs taken to get the runs
#made by each team in each year in each of the cricket formats. It also tallies number of wickets fallen

from collections import defaultdict
import json
import os
import io

test_df.name = 'test_df'
odi_df.name = 'odi_df'
t20_df.name = 't20_df'

for formatx in [test_df, odi_df, t20_df]:

    jdict = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
    
    for page in formatx.keys():
        print page
        year = formatx[page]['year']
        try:
            formatx[page]['innings'].keys()
        except:
            continue
        else:
            for inning in formatx[page]['innings'].keys():
                for team in formatx[page]['innings'][inning]['team_batting'].keys():
                    if team in nation_list:
                        try:
                            int(formatx[page]['innings'][inning]['team_batting'][team]['total']['runs'])
                        except:
                            continue
                        else:
                            jdict[team][year]['runs'] += float(formatx[page]['innings'][inning]['team_batting']\
                                                             [team]['total']['runs'])
                            jdict[team][year]['overs'] += float(formatx[page]['innings'][inning]['team_batting']\
                                                              [team]['total']['overs'])
                            jdict[team][year]['wickets'] += float(formatx[page]['innings'][inning]['team_batting']\
                                                              [team]['total']['wickets'])
                                                                        
    jstring = json.dumps(jdict, indent=4, sort_keys=True)
    
    filenameg = formatx.name + '_overs_runs_wickets_by_year.json' 
    fnameg = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenameg)

    with io.open(fnameg, 'w') as json_file:
        json_file.write(unicode(jstring))

print "Processing done"

In [ ]:
#this block is to get the maidens and overs bowled by
#each team in each year in each of the cricket formats

from collections import defaultdict
import json
import os
import io


test_df.name = 'test_df'
odi_df.name = 'odi_df'
t20_df.name = 't20_df'

for formatx in [test_df, odi_df, t20_df]:

    jdict = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
    
    for page in formatx.keys():
        print page
        year = formatx[page]['year']
        try:
            formatx[page]['innings'].keys()
        except:
            continue
        else:
            for inning in formatx[page]['innings'].keys():
                try:
                    formatx[page]['innings'][inning]['team_bowling'].keys()
                except:
                    continue
                else:
                    for team in formatx[page]['innings'][inning]['team_bowling'].keys():
                        if team in nation_list:
                            for bowler in formatx[page]['innings'][inning]['team_bowling'][team]['bowlers_all'].keys():
                                jdict[team][year]['overs'] += float(formatx[page]['innings'][inning]['team_bowling']\
                                                                  [team]['bowlers_all'][bowler]['overs'])
                                try:
                                    float(formatx[page]['innings'][inning]['team_bowling'][team]['bowlers_all'][bowler]['maidens'])
                                except:
                                    continue
                                else:
                                    jdict[team][year]['maidens'] += float(formatx[page]['innings'][inning]['team_bowling']\
                                                                 [team]['bowlers_all'][bowler]['maidens'])
                            
                                                                        
    jstring = json.dumps(jdict, indent=4, sort_keys=True)
    
    filenameg = formatx.name + '_maidens_overs_by_year.json' 
    fnameg = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenameg)

    with io.open(fnameg, 'w') as json_file:
        json_file.write(unicode(jstring))

print "Processing done"

In [10]:
#this block is to get the number of boundaries, sixes and overs distributed by year and team
#individual scores made in all innings considered for this

from collections import defaultdict
import json
import os
import io

#trial_df = pd.read_json('trial3.json') #loading a smaller json just to check if the code's working

#trial_df.name = 'trial_df'

test_df.name = 'test_df'
odi_df.name = 'odi_df'
t20_df.name = 't20_df'

#for formatx in [trial_df]:
for formatx in [test_df, odi_df, t20_df]:

    jdict = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
    
    for page in formatx.keys():
        print page
        year = formatx[page]['year']
        try:
            formatx[page]['innings'].keys()
        except:
            continue
        else:
            for inning in formatx[page]['innings'].keys():
                
                for team in formatx[page]['innings'][inning]['team_batting'].keys():
                    if team in nation_list:
                        try:
                            int(formatx[page]['innings'][inning]['team_batting'][team]['total']['runs'])
                        except:
                            continue
                        else:
                            overs = float(formatx[page]['innings'][inning]['team_batting'][team]['total']['overs'])
                            jdict[team][year]['overs'] += overs

                            for order in formatx[page]['innings'][inning]['team_batting'][team]['batsman_all'].keys():
                                for namex in formatx[page]['innings'][inning]['team_batting'][team]['batsman_all'][order].keys():

                                    try:
                                        int(formatx[page]['innings'][inning]['team_batting'][team]['batsman_all'][order]\
                                           [namex]['batsman_4s'])
                                    except:
                                        pass
                                    else:
                                        jdict[team][year]['fours'] += int(formatx[page]['innings'][inning]['team_batting']\
                                                                          [team]['batsman_all'][order][namex]['batsman_4s'])

                                    try:
                                        int(formatx[page]['innings'][inning]['team_batting'][team]['batsman_all'][order]\
                                           [namex]['batsman_6s'])
                                    except:
                                        continue
                                    else:
                                        jdict[team][year]['sixes'] += int(formatx[page]['innings'][inning]['team_batting']\
                                                                          [team]['batsman_all'][order][namex]['batsman_6s'])

                                            
    jstring = json.dumps(jdict, indent=4, sort_keys=True)
    
    filenameg = formatx.name + '_fours_sixes_overs_by_year.json' 
    fnameg = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenameg)

    with io.open(fnameg, 'w') as json_file:
        json_file.write(unicode(jstring))

print "Processing done"

test_0001.html
test_0002.html
.
.
.
t20_0617.html
Processing done


In [14]:
#this block is to find out how often a run out happens, how many overs on average between someone being run out

from collections import defaultdict
import json
import os
import io

test_df.name = 'test_df'
odi_df.name = 'odi_df'
t20_df.name = 't20_df'

for formatx in [test_df, odi_df, t20_df]:

    jdict = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
    
    for page in formatx.keys():
        print page
        year = formatx[page]['year']
        try:
            formatx[page]['innings'].keys()
        except:
            continue
        else:
            for inning in formatx[page]['innings'].keys():
                for team in formatx[page]['innings'][inning]['team_batting'].keys():
                    if team in nation_list:
                        try:
                            int(formatx[page]['innings'][inning]['team_batting'][team]['total']['runs'])
                        except:
                            continue
                        else:
                            jdict[team][year]['runs'] += float(formatx[page]['innings'][inning]['team_batting']\
                                                             [team]['total']['runs'])
                            jdict[team][year]['overs'] += float(formatx[page]['innings'][inning]['team_batting']\
                                                              [team]['total']['overs'])
                            jdict[team][year]['wickets'] += float(formatx[page]['innings'][inning]['team_batting']\
                                                              [team]['total']['wickets'])

                            for order in formatx[page]['innings'][inning]['team_batting'][team]['batsman_all'].keys():
                                for namex in formatx[page]['innings'][inning]['team_batting'][team]['batsman_all'][order].keys():
                                    try:
                                        formatx[page]['innings'][inning]['team_batting'][team]['batsman_all'][order]\
                                           [namex]['batsman_dismissal_text']
                                    except:
                                        continue
                                    else:
                                        if 'run out' in formatx[page]['innings'][inning]['team_batting'][team]['batsman_all'][order]\
                                           [namex]['batsman_dismissal_text']:
                                                jdict[team][year]['run_outs'] += 1                                    
                                    
    jstring = json.dumps(jdict, indent=4, sort_keys=True)
    
    filenameg = formatx.name + '_run_outs_by_year.json' 
    fnameg = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenameg)

    with io.open(fnameg, 'w') as json_file:
        json_file.write(unicode(jstring))

print "Processing done"

test_0001.html
test_0002.html
.
.
.
t20_0616.html
t20_0617.html
Processing done


In [15]:
#this block is to find out how often a draw happens in test matches

from collections import defaultdict
import json
import os
import io

test_df.name = 'test_df'

for formatx in [test_df]:

    jdict = defaultdict(lambda: defaultdict(int))
    
    for page in formatx.keys():
        print page
        year = formatx[page]['year']
        try:
            formatx[page]['result_text']
        except:
            continue
        else:
            if 'won by' in formatx[page]['result_text']:
                jdict[year]['win_loss'] += 1
            elif 'drawn' in formatx[page]['result_text']:
                jdict[year]['draw'] += 1
            else:
                jdict[year]['other_result'] += 1
                                    
    jstring = json.dumps(jdict, indent=4, sort_keys=True)
    
    filenameg = formatx.name + '_draws_by_year.json' 
    fnameg = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenameg)

    with io.open(fnameg, 'w') as json_file:
        json_file.write(unicode(jstring))

print "Processing done"

test_0001.html
.
.
.
test_1124.html
test_1125.html
Processing done


In [119]:
#this is some further processing of the data we'd extracted earlier. So we'd earlier gotten how many totals between
#0-200, 200-400 etc. were scored by each team in a year. Here what we're doing is creating groups of five years each
#say 1997-2001, 2002-2006 etc. and getting the figures for these five-year groups

#am also creating a separate category that just totals up the figures for the 10 test-playing nations

#also i'm taking only the last twenty years 1997 to 2016 for this

from collections import defaultdict
import json
import os
import io
import numpy as np

#test_df_totals = pd.read_json('test_df_totals_by_year.json')
#didnt' use pandas to load the json here, pandas was creating keys where there weren't any and was giving me lots of errors
#used the basic json python library instead

with open('test_df_totals_by_year.json', 'r') as f:
     test_df_totals = json.load(f)

jdict = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))


first_band = ["1997","1998","1999","2000","2001"]
second_band = ["2002","2003","2004","2005","2006"]
third_band = ["2007","2008","2009","2010","2011"]
fourth_band = ["2012","2013","2014","2015","2016"]


for nation in test_df_totals.keys():
    if nation in ['India','Pakistan','Sri Lanka']:
        for year in test_df_totals[nation].keys():
            #year = year.astype(np.int64)
            if year in first_band:
                for rangex in test_df_totals[nation][year].keys():
                    jdict[nation]['1997-2001'][rangex] += test_df_totals[nation][year][rangex]
            elif year in second_band:
                for rangex in test_df_totals[nation][year].keys():
                    jdict[nation]['2002-2006'][rangex] += test_df_totals[nation][year][rangex]
            elif year in third_band:
                #year = str(year)
                for rangex in test_df_totals[nation][year].keys():
                    jdict[nation]['2007-2011'][rangex] += test_df_totals[nation][year][rangex]
            elif year in fourth_band:
                for rangex in test_df_totals[nation][year].keys():
                    jdict[nation]['2012-2016'][rangex] += test_df_totals[nation][year][rangex]

for nation in test_df_totals.keys():
    for year in test_df_totals[nation].keys():
        #year = year.astype(np.int64)
        if year in first_band:
            for rangex in test_df_totals[nation][year].keys():
                jdict['Test_nations']['1997-2001'][rangex] += test_df_totals[nation][year][rangex]
        elif year in second_band:
            for rangex in test_df_totals[nation][year].keys():
                jdict['Test_nations']['2002-2006'][rangex] += test_df_totals[nation][year][rangex]
        elif year in third_band:
            for rangex in test_df_totals[nation][year].keys():
                jdict['Test_nations']['2007-2011'][rangex] += test_df_totals[nation][year][rangex]
        elif year in fourth_band:
            for rangex in test_df_totals[nation][year].keys():
                jdict['Test_nations']['2012-2016'][rangex] += test_df_totals[nation][year][rangex]


jstring = json.dumps(jdict, indent=4, sort_keys=True)

filenameg = 'final_test_totals.json' 
fnameg = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenameg)

with io.open(fnameg, 'w') as json_file:
    json_file.write(unicode(jstring))

print "Processing done"

Processing done


In [120]:
#same purpose as the code block above, except this is for ODIs

with open('odi_df_totals_by_year.json', 'r') as f:
     odi_df_totals = json.load(f)

jdict = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))


first_band = ["1997","1998","1999","2000","2001"]
second_band = ["2002","2003","2004","2005","2006"]
third_band = ["2007","2008","2009","2010","2011"]
fourth_band = ["2012","2013","2014","2015","2016"]


for nation in odi_df_totals.keys():
    if nation in ['India','Pakistan','Sri Lanka']:
        for year in odi_df_totals[nation].keys():
            #year = year.astype(np.int64)
            if year in first_band:
                for rangex in odi_df_totals[nation][year].keys():
                    jdict[nation]['1997-2001'][rangex] += odi_df_totals[nation][year][rangex]
            elif year in second_band:
                for rangex in odi_df_totals[nation][year].keys():
                    jdict[nation]['2002-2006'][rangex] += odi_df_totals[nation][year][rangex]
            elif year in third_band:
                #year = str(year)
                for rangex in odi_df_totals[nation][year].keys():
                    jdict[nation]['2007-2011'][rangex] += odi_df_totals[nation][year][rangex]
            elif year in fourth_band:
                for rangex in odi_df_totals[nation][year].keys():
                    jdict[nation]['2012-2016'][rangex] += odi_df_totals[nation][year][rangex]

for nation in odi_df_totals.keys():
    for year in odi_df_totals[nation].keys():
        if year in first_band:
            for rangex in odi_df_totals[nation][year].keys():
                jdict['Test_nations']['1997-2001'][rangex] += odi_df_totals[nation][year][rangex]
        elif year in second_band:
            for rangex in odi_df_totals[nation][year].keys():
                jdict['Test_nations']['2002-2006'][rangex] += odi_df_totals[nation][year][rangex]
        elif year in third_band:
            for rangex in odi_df_totals[nation][year].keys():
                jdict['Test_nations']['2007-2011'][rangex] += odi_df_totals[nation][year][rangex]
        elif year in fourth_band:
            for rangex in odi_df_totals[nation][year].keys():
                jdict['Test_nations']['2012-2016'][rangex] += odi_df_totals[nation][year][rangex]


jstring = json.dumps(jdict, indent=4, sort_keys=True)

filenameg = 'final_odi_totals.json' 
fnameg = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenameg)

with io.open(fnameg, 'w') as json_file:
    json_file.write(unicode(jstring))

print "Processing done"

Processing done


In [121]:
#this takes the number of centuries, double and triple centuries scored by individual batsmen in each year
#totals them up for each five-year group 1997-2001 , 2002-2006 etc.
#it also does it for all the Test nations

with open('test_df_indiv_scores_by_year.json', 'r') as f:
     test_df_indiv_scores = json.load(f)

jdict = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))


first_band = ["1997","1998","1999","2000","2001"]
second_band = ["2002","2003","2004","2005","2006"]
third_band = ["2007","2008","2009","2010","2011"]
fourth_band = ["2012","2013","2014","2015","2016"]


for nation in test_df_indiv_scores.keys():
    if nation in ['India','Pakistan','Sri Lanka']:
        for year in test_df_indiv_scores[nation].keys():
            if year in first_band:
                for rangex in test_df_indiv_scores[nation][year].keys():
                    if rangex != '0-99':
                        jdict[nation]['1997-2001'][rangex] += test_df_indiv_scores[nation][year][rangex]
            elif year in second_band:
                for rangex in test_df_indiv_scores[nation][year].keys():
                    if rangex != '0-99':
                        jdict[nation]['2002-2006'][rangex] += test_df_indiv_scores[nation][year][rangex]
            elif year in third_band:
                for rangex in test_df_indiv_scores[nation][year].keys():
                    if rangex != '0-99':
                        jdict[nation]['2007-2011'][rangex] += test_df_indiv_scores[nation][year][rangex]
            elif year in fourth_band:
                for rangex in test_df_indiv_scores[nation][year].keys():
                    if rangex != '0-99':
                        jdict[nation]['2012-2016'][rangex] += test_df_indiv_scores[nation][year][rangex]

for nation in test_df_indiv_scores.keys():
    for year in test_df_indiv_scores[nation].keys():
        if year in first_band:
            for rangex in test_df_indiv_scores[nation][year].keys():
                if rangex != '0-99':
                    jdict['Test_nations']['1997-2001'][rangex] += test_df_indiv_scores[nation][year][rangex]
        elif year in second_band:
            for rangex in test_df_indiv_scores[nation][year].keys():
                if rangex != '0-99':
                    jdict['Test_nations']['2002-2006'][rangex] += test_df_indiv_scores[nation][year][rangex]
        elif year in third_band:
            for rangex in test_df_indiv_scores[nation][year].keys():
                if rangex != '0-99':
                    jdict['Test_nations']['2007-2011'][rangex] += test_df_indiv_scores[nation][year][rangex]
        elif year in fourth_band:
            for rangex in test_df_indiv_scores[nation][year].keys():
                if rangex != '0-99':
                    jdict['Test_nations']['2012-2016'][rangex] += test_df_indiv_scores[nation][year][rangex]


jstring = json.dumps(jdict, indent=4, sort_keys=True)

filenameg = 'final_test_indiv_scores.json' 
fnameg = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenameg)

with io.open(fnameg, 'w') as json_file:
    json_file.write(unicode(jstring))

print "Processing done"

Processing done


In [123]:
#same motivation as the code block above, just that this is done for ODIs

with open('odi_df_indiv_scores_by_year.json', 'r') as f:
     odi_df_indiv_scores = json.load(f)

jdict = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))

first_band = ["1997","1998","1999","2000","2001"]
second_band = ["2002","2003","2004","2005","2006"]
third_band = ["2007","2008","2009","2010","2011"]
fourth_band = ["2012","2013","2014","2015","2016"]

for nation in odi_df_indiv_scores.keys():
    if nation in ['India','Pakistan','Sri Lanka']:
        for year in odi_df_indiv_scores[nation].keys():
            if year in first_band:
                for rangex in odi_df_indiv_scores[nation][year].keys():
                    if rangex != '0-49':
                        jdict[nation]['1997-2001'][rangex] += odi_df_indiv_scores[nation][year][rangex]
            elif year in second_band:
                for rangex in odi_df_indiv_scores[nation][year].keys():
                    if rangex != '0-49':
                        jdict[nation]['2002-2006'][rangex] += odi_df_indiv_scores[nation][year][rangex]
            elif year in third_band:
                for rangex in odi_df_indiv_scores[nation][year].keys():
                    if rangex != '0-49':
                        jdict[nation]['2007-2011'][rangex] += odi_df_indiv_scores[nation][year][rangex]
            elif year in fourth_band:
                for rangex in odi_df_indiv_scores[nation][year].keys():
                    if rangex != '0-49':
                        jdict[nation]['2012-2016'][rangex] += odi_df_indiv_scores[nation][year][rangex]

for nation in odi_df_indiv_scores.keys():
    for year in odi_df_indiv_scores[nation].keys():
        if year in first_band:
            for rangex in odi_df_indiv_scores[nation][year].keys():
                if rangex != '0-49':
                    jdict['test_nations']['1997-2001'][rangex] += odi_df_indiv_scores[nation][year][rangex]
        elif year in second_band:
            for rangex in odi_df_indiv_scores[nation][year].keys():
                if rangex != '0-49':
                    jdict['test_nations']['2002-2006'][rangex] += odi_df_indiv_scores[nation][year][rangex]
        elif year in third_band:
            for rangex in odi_df_indiv_scores[nation][year].keys():
                if rangex != '0-49':
                    jdict['test_nations']['2007-2011'][rangex] += odi_df_indiv_scores[nation][year][rangex]
        elif year in fourth_band:
            for rangex in odi_df_indiv_scores[nation][year].keys():
                if rangex != '0-49':
                    jdict['test_nations']['2012-2016'][rangex] += odi_df_indiv_scores[nation][year][rangex]


jstring = json.dumps(jdict, indent=4, sort_keys=True)

filenameg = 'final_odi_indiv_scores.json' 
fnameg = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenameg)

with io.open(fnameg, 'w') as json_file:
    json_file.write(unicode(jstring))

print "Processing done"

Processing done


In [130]:
#while I was doing this code block, I thought of having four separate categories, India, Pakistan, Sri Lanka and
# 'all test nations together' . Decided later against having more than one category as it would have just cluttered up
#my graphics.

#but anyway what this code is create a json file with the number overs bowled in a year, the no. of wickets fallen and 
#number of runs scored. So this would tell you the rate at which runs have been scored in a year or the frequency
#with which wickets have fallen


with open('test_df_overs_runs_wickets_by_year.json', 'r') as f:
     test_df_run_wicket_rate = json.load(f)

jdict = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))


for nation in test_df_run_wicket_rate.keys():
    if nation in ['India','Pakistan','Sri Lanka']:
        for year in test_df_run_wicket_rate[nation].keys():
            if year not in ['1990','2017']:
                jdict[nation][year]['run_rate'] = (test_df_run_wicket_rate[nation][year]['runs']/test_df_run_wicket_rate[nation]\
                    [year]['overs'])*90
                jdict[nation][year]['wicket_rate'] = test_df_run_wicket_rate[nation][year]['overs']/\
                                                    test_df_run_wicket_rate[nation][year]['wickets']


for nation in test_df_run_wicket_rate.keys():
    for year in test_df_run_wicket_rate[nation].keys():
        if year not in ['1990','2017']:
            jdict['test_nations'][year]['runs'] += test_df_run_wicket_rate[nation][year]['runs']
            jdict['test_nations'][year]['wickets'] += test_df_run_wicket_rate[nation][year]['wickets']
            jdict['test_nations'][year]['overs'] += test_df_run_wicket_rate[nation][year]['overs']
            jdict['test_nations'][year]['wicket_rate'] = jdict['test_nations'][year]['overs']/jdict['test_nations'][year]['wickets']
            jdict['test_nations'][year]['run_rate'] =(jdict['test_nations'][year]['runs']/jdict['test_nations'][year]['overs'])*90

jstring = json.dumps(jdict, indent=4, sort_keys=True)

filenameg = 'final_test_run_wicket_rate.json' 
fnameg = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenameg)

with io.open(fnameg, 'w') as json_file:
    json_file.write(unicode(jstring))

print "Processing done"

Processing done


In [129]:
#same motivation as the code block above, just that it's for ODIs

with open('odi_df_overs_runs_wickets_by_year.json', 'r') as f:
     odi_df_run_wicket_rate = json.load(f)

jdict = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))

for nation in odi_df_run_wicket_rate.keys():
    if nation in ['India','Pakistan','Sri Lanka']:
        for year in odi_df_run_wicket_rate[nation].keys():
            if year not in ['1990','2017']:
                jdict[nation][year]['run_rate'] = (odi_df_run_wicket_rate[nation][year]['runs']/odi_df_run_wicket_rate[nation]\
                    [year]['overs'])*50
                jdict[nation][year]['wicket_rate'] = odi_df_run_wicket_rate[nation][year]['overs']/odi_df_run_wicket_rate[nation]\
                                                      [year]['wickets']
                #this looks at how often a wicket falls, ever xx overs, you don't need to *90 for this


for nation in odi_df_run_wicket_rate.keys():
    for year in odi_df_run_wicket_rate[nation].keys():
        if year not in ['1990','2017']:
            jdict['test_nations'][year]['runs'] += odi_df_run_wicket_rate[nation][year]['runs']
            jdict['test_nations'][year]['wickets'] += odi_df_run_wicket_rate[nation][year]['wickets']
            jdict['test_nations'][year]['overs'] += odi_df_run_wicket_rate[nation][year]['overs']
            jdict['test_nations'][year]['wicket_rate'] = jdict['test_nations'][year]['overs']/jdict['test_nations'][year]['wickets']
            jdict['test_nations'][year]['run_rate'] =(jdict['test_nations'][year]['runs']/jdict['test_nations'][year]['overs'])*50

jstring = json.dumps(jdict, indent=4, sort_keys=True)

filenameg = 'final_odi_run_wicket_rate.json' 
fnameg = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenameg)

with io.open(fnameg, 'w') as json_file:
    json_file.write(unicode(jstring))

print "Processing done"

Processing done


In [175]:
#this looks at what percent of overs in a year are maiden overs

with open('test_df_maidens_overs_by_year.json', 'r') as f:
     test_df_maidens = json.load(f)

jdict = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))

for nation in test_df_maidens.keys():
    if nation in ['India','Pakistan','Sri Lanka']:
        for year in test_df_maidens[nation].keys():
            if year not in ['1990','2017']:
                jdict[nation][year]['maiden_rate'] =\
                    (test_df_maidens[nation][year]['maidens']/test_df_maidens[nation][year]['overs'])*100

                


for nation in test_df_maidens.keys():
    for year in test_df_maidens[nation].keys():
        if year not in ['1990','2017']:
            jdict['test_nations'][year]['maidens'] += test_df_maidens[nation][year]['maidens']
            jdict['test_nations'][year]['overs'] += test_df_maidens[nation][year]['overs']
            jdict['test_nations'][year]['maiden_rate'] =\
                (jdict['test_nations'][year]['maidens']/jdict['test_nations'][year]['overs'])*100

jstring = json.dumps(jdict, indent=4, sort_keys=True)

filenameg = 'final_test_maidens_rate.json' 
fnameg = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenameg)

with io.open(fnameg, 'w') as json_file:
    json_file.write(unicode(jstring))

print "Processing done"

Processing done


In [177]:
#same purpose as the code block above, just that this is just for ODIs

with open('odi_df_maidens_overs_by_year.json', 'r') as f:
     odi_df_maidens = json.load(f)

jdict = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))

for nation in odi_df_maidens.keys():
    if nation in ['India','Pakistan','Sri Lanka']:
        for year in odi_df_maidens[nation].keys():
            if year not in ['1990','2017']:
                jdict[nation][year]['maiden_rate'] =\
                    (odi_df_maidens[nation][year]['maidens']/odi_df_maidens[nation][year]['overs'])*100



for nation in odi_df_maidens.keys():
    for year in odi_df_maidens[nation].keys():
        if year not in ['1990','2017']:
            jdict['test_nations'][year]['maidens'] += odi_df_maidens[nation][year]['maidens']
            jdict['test_nations'][year]['overs'] += odi_df_maidens[nation][year]['overs']
            jdict['test_nations'][year]['maiden_rate'] =\
                (jdict['test_nations'][year]['maidens']/jdict['test_nations'][year]['overs'])*100

jstring = json.dumps(jdict, indent=4, sort_keys=True)

filenameg = 'final_odi_maidens_rate.json' 
fnameg = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenameg)

with io.open(fnameg, 'w') as json_file:
    json_file.write(unicode(jstring))

print "Processing done"

Processing done


In [143]:
#this tell you how many fours and sixes have been scored in test matches in a year

with open('test_df_fours_sixes_overs_by_year.json', 'r') as f:
     test_df_fours_sixes = json.load(f)

jdict = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))

for nation in test_df_fours_sixes.keys():
    if nation in ['India','Pakistan','Sri Lanka']:
        for year in test_df_fours_sixes[nation].keys():
            if year not in ['1990','2017']:
                jdict[nation][year]['fours_rate'] = \
                    test_df_fours_sixes[nation][year]['overs']/test_df_fours_sixes[nation][year]['fours']
                if 'sixes' in test_df_fours_sixes[nation][year].keys():
                    jdict[nation][year]['sixes_rate'] = \
                        test_df_fours_sixes[nation][year]['overs']/test_df_fours_sixes[nation][year]['sixes']

                #this looks at how often we see a four or a six, every xx overs

for nation in test_df_fours_sixes.keys():
    for year in test_df_fours_sixes[nation].keys():
        if year not in ['1990','2017']:
            jdict['test_nations'][year]['fours'] += test_df_fours_sixes[nation][year]['fours']
            jdict['test_nations'][year]['overs'] += test_df_fours_sixes[nation][year]['overs']
            jdict['test_nations'][year]['fours_rate'] = jdict['test_nations'][year]['overs']/jdict['test_nations'][year]['fours']
            if 'sixes' in test_df_fours_sixes[nation][year].keys():
                jdict['test_nations'][year]['sixes'] += test_df_fours_sixes[nation][year]['sixes']
                jdict['test_nations'][year]['sixes_rate'] = jdict['test_nations'][year]['overs']/jdict['test_nations'][year]['sixes']

jstring = json.dumps(jdict, indent=4, sort_keys=True)

filenameg = 'final_test_four_six_rate.json' 
fnameg = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenameg)

with io.open(fnameg, 'w') as json_file:
    json_file.write(unicode(jstring))

print "Processing done"

Processing done


In [144]:
#same purpose as the code block above, just that this is for ODIs

with open('odi_df_fours_sixes_overs_by_year.json', 'r') as f:
     odi_df_fours_sixes = json.load(f)

jdict = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))

for nation in odi_df_fours_sixes.keys():
    if nation in ['India','Pakistan','Sri Lanka']:
        for year in odi_df_fours_sixes[nation].keys():
            if year not in ['1990','2017']:
                if 'fours' in odi_df_fours_sixes[nation][year].keys():
                    jdict[nation][year]['fours_rate'] = \
                        odi_df_fours_sixes[nation][year]['overs']/odi_df_fours_sixes[nation][year]['fours']
                if 'sixes' in odi_df_fours_sixes[nation][year].keys():
                    jdict[nation][year]['sixes_rate'] = \
                        odi_df_fours_sixes[nation][year]['overs']/odi_df_fours_sixes[nation][year]['sixes']

                #this looks at how often we see a four or a six, every xx overs

for nation in odi_df_fours_sixes.keys():
    for year in odi_df_fours_sixes[nation].keys():
        if year not in ['1990','2017']:
            jdict['test_nations'][year]['fours'] += odi_df_fours_sixes[nation][year]['fours']
            jdict['test_nations'][year]['overs'] += odi_df_fours_sixes[nation][year]['overs']
            jdict['test_nations'][year]['fours_rate'] = jdict['test_nations'][year]['overs']/jdict['test_nations'][year]['fours']
            if 'sixes' in odi_df_fours_sixes[nation][year].keys():
                jdict['test_nations'][year]['sixes'] += odi_df_fours_sixes[nation][year]['sixes']
                jdict['test_nations'][year]['sixes_rate'] = jdict['test_nations'][year]['overs']/jdict['test_nations'][year]['sixes']

jstring = json.dumps(jdict, indent=4, sort_keys=True)

filenameg = 'final_odi_four_six_rate.json' 
fnameg = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenameg)

with io.open(fnameg, 'w') as json_file:
    json_file.write(unicode(jstring))

print "Processing done"

Processing done


In [178]:
#this looks at what percentage of test match dismissals are run outs in a particular year

with open('test_df_run_outs_by_year.json', 'r') as f:
     test_df_run_outs = json.load(f)

jdict = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))

for nation in test_df_run_outs.keys():
    if nation in ['India','Pakistan','Sri Lanka']:
        for year in test_df_run_outs[nation].keys():
            if year not in ['1990','2017']:
                if 'run_outs' in test_df_run_outs[nation][year].keys():
                    jdict[nation][year]['run_out_rate'] = \
                        (test_df_run_outs[nation][year]['run_outs']/test_df_run_outs[nation][year]['wickets'])*100
                


for nation in test_df_run_outs.keys():
    #print nation
    for year in test_df_run_outs[nation].keys():
        if year not in ['1990','2017']:
            jdict['test_nations'][year]['wickets'] += test_df_run_outs[nation][year]['wickets']
            #print year
            try:
                test_df_run_outs[nation][year]['run_outs']
            except:
                continue
            else:
                jdict['test_nations'][year]['run_outs'] += test_df_run_outs[nation][year]['run_outs']
                jdict['test_nations'][year]['run_out_rate'] = \
                    (jdict['test_nations'][year]['run_outs']/jdict['test_nations'][year]['wickets'])*100

jstring = json.dumps(jdict, indent=4, sort_keys=True)

filenameg = 'final_test_run_out_rate.json' 
fnameg = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenameg)

with io.open(fnameg, 'w') as json_file:
    json_file.write(unicode(jstring))

print "Processing done"

Processing done


In [179]:
#same purpose as the code block above, just that this is for ODIs

with open('odi_df_run_outs_by_year.json', 'r') as f:
     odi_df_run_outs = json.load(f)

jdict = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))

for nation in odi_df_run_outs.keys():
    if nation in ['India','Pakistan','Sri Lanka']:
        for year in odi_df_run_outs[nation].keys():
            if year not in ['1990','2017']:
                if 'run_outs' in odi_df_run_outs[nation][year].keys():
                    jdict[nation][year]['run_out_rate'] = \
                        (odi_df_run_outs[nation][year]['run_outs']/odi_df_run_outs[nation][year]['wickets'])*100
                


for nation in odi_df_run_outs.keys():
    for year in odi_df_run_outs[nation].keys():
        if year not in ['1990','2017']:
            jdict['test_nations'][year]['wickets'] += odi_df_run_outs[nation][year]['wickets']
            if 'run_outs' in odi_df_run_outs[nation][year].keys():
                jdict['test_nations'][year]['run_outs'] += odi_df_run_outs[nation][year]['run_outs']
                jdict['test_nations'][year]['run_out_rate'] = \
                    (jdict['test_nations'][year]['run_outs']/jdict['test_nations'][year]['wickets'])*100

jstring = json.dumps(jdict, indent=4, sort_keys=True)

filenameg = 'final_odi_run_out_rate.json' 
fnameg = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenameg)

with io.open(fnameg, 'w') as json_file:
    json_file.write(unicode(jstring))

print "Processing done"

Processing done


In [154]:
#this looks at what percentage of test matches in each year are drawn

with open('test_df_draws_by_year.json', 'r') as f:
     test_df_draws = json.load(f)

jdict = defaultdict(lambda: defaultdict(float))

for year in test_df_draws.keys():
    if year not in ['1990','2017']:
        for result in test_df_draws[year].keys():
            jdict[year]['total_tests'] += test_df_draws[year][result]
            
            jdict[year]['draw_percent'] = (test_df_draws[year]['draw']/jdict[year]['total_tests'])*100
            
            #note that the draw percent is actually calculated as many times as there are 'result' in [year].keys
            #it's ok this time, as the result was still calculated correctly, but you might want to keep that out of the 
            #loop next time
        
jstring = json.dumps(jdict, indent=4, sort_keys=True)

filenameg = 'final_draw_percent.json' 
fnameg = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenameg)

with io.open(fnameg, 'w') as json_file:
    json_file.write(unicode(jstring))

print "Processing done"

Processing done


In [169]:
#again some more processing of the files, by this point I've decided to not use figures for separate countries but instead
# just focus on the aggregate figures for 'test nations' as a whole

#This isn't even the final files I ended up using, I created another set of json files from this, with just a few minor changes
#from these json files, and then another set of CSV files which are the ones I finally used to create the article's graphics

with open('final_test_totals.json', 'r') as f:
     test_totals_again = json.load(f)

jdict = defaultdict(lambda: defaultdict(lambda: defaultdict(float)))

for nation in test_totals_again.keys():
    #print nation
    for year_band in test_totals_again[nation].keys():
        #print year_band
        for score_range in test_totals_again[nation][year_band].keys():
            #print score_range
            jdict[nation][year_band]['total_innings'] += test_totals_again[nation][year_band][score_range]
        for score_range in test_totals_again[nation][year_band].keys():
            jdict[nation][year_band][score_range] =\
                (test_totals_again[nation][year_band][score_range]/jdict[nation][year_band]['total_innings'])*100
        
jstring = json.dumps(jdict, indent=4, sort_keys=True)

filenameg = 'final_test_totals_percent.json' 
fnameg = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenameg)

with io.open(fnameg, 'w') as json_file:
    json_file.write(unicode(jstring))

print "Processing done"

Processing done


In [170]:
#same purpose as the code block above, just that this is for ODIs

with open('final_odi_totals.json', 'r') as f:
     odi_totals_again = json.load(f)

jdict = defaultdict(lambda: defaultdict(lambda: defaultdict(float)))

for nation in odi_totals_again.keys():
    #print nation
    for year_band in odi_totals_again[nation].keys():
        #print year_band
        for score_range in odi_totals_again[nation][year_band].keys():
            #print score_range
            jdict[nation][year_band]['total_innings'] += odi_totals_again[nation][year_band][score_range]
        for score_range in odi_totals_again[nation][year_band].keys():
            jdict[nation][year_band][score_range] =\
                (odi_totals_again[nation][year_band][score_range]/jdict[nation][year_band]['total_innings'])*100
        
jstring = json.dumps(jdict, indent=4, sort_keys=True)

filenameg = 'final_odi_totals_percent.json' 
fnameg = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenameg)

with io.open(fnameg, 'w') as json_file:
    json_file.write(unicode(jstring))

print "Processing done"

Processing done


In [180]:
#i didn't end up using this file, but this looks at what % of all first innings test totals were in particular
#score range, 0-200, 201-400 etc.

with open('final_test_totals_percent.json', 'r') as f:
     test_percent = json.load(f)
        
with open('final_test_totals.json', 'r') as g:
     test_totals = json.load(g)

jdict = defaultdict(lambda: defaultdict(lambda: defaultdict(float)))

for year_band in test_percent['Test_nations'].keys():
    jdict[year_band]['0-200']['percent'] = test_percent['Test_nations'][year_band]['0-200']
    jdict[year_band]['201-400']['percent'] = test_percent['Test_nations'][year_band]['201-400']
    jdict[year_band]['401-600']['percent'] = test_percent['Test_nations'][year_band]['401-600']
    jdict[year_band]['600-plus']['percent'] = test_percent['Test_nations'][year_band]['600 plus']

for year_band in test_totals['Test_nations'].keys():
    jdict[year_band]['0-200']['total'] = test_totals['Test_nations'][year_band]['0-200']
    jdict[year_band]['201-400']['total'] = test_totals['Test_nations'][year_band]['201-400']
    jdict[year_band]['401-600']['total'] = test_totals['Test_nations'][year_band]['401-600']
    jdict[year_band]['600-plus']['total'] = test_totals['Test_nations'][year_band]['600 plus']
    

jstring = json.dumps(jdict, indent=4, sort_keys=True)

filenameg = 'again_test_totals.json' 
fnameg = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenameg)

with io.open(fnameg, 'w') as json_file:
    json_file.write(unicode(jstring))

print "Processing done"

Processing done


In [182]:
#calculates the no. of tests between a double or a triple century on average
# decided later to not use this metric


with open('final_test_indiv_scores.json', 'r') as f:
     test_indiv = json.load(f)
        
with open('final_draw_percent.json', 'r') as g:
     test_sum = json.load(g)

jdict = defaultdict(lambda: defaultdict(lambda: defaultdict(float)))

first_band = ["1997","1998","1999","2000","2001"]
second_band = ["2002","2003","2004","2005","2006"]
third_band = ["2007","2008","2009","2010","2011"]
fourth_band = ["2012","2013","2014","2015","2016"]

for year in test_sum.keys():
    if year in first_band:
        jdict['1997-2001']['200-299']['tests'] += test_sum[year]['total_tests']
        jdict['1997-2001']['300 & over']['tests'] += test_sum[year]['total_tests']
    elif year in second_band:
        jdict['2002-2006']['200-299']['tests'] += test_sum[year]['total_tests']
        jdict['2002-2006']['300 & over']['tests'] += test_sum[year]['total_tests']
    elif year in third_band:
        jdict['2007-2011']['200-299']['tests'] += test_sum[year]['total_tests']
        jdict['2007-2011']['300 & over']['tests'] += test_sum[year]['total_tests']
    elif year in fourth_band:
        jdict['2012-2016']['200-299']['tests'] += test_sum[year]['total_tests']
        jdict['2012-2016']['300 & over']['tests'] += test_sum[year]['total_tests']
        
for year_band in test_indiv['Test_nations'].keys():        
    jdict[year_band]['200-299']['total'] = test_indiv['Test_nations'][year_band]['200-299']
    jdict[year_band]['200-299']['per_tests'] =\
        jdict[year_band]['200-299']['tests']/jdict[year_band]['200-299']['total']
    
    jdict[year_band]['300 & over']['total'] = test_indiv['Test_nations'][year_band]['300 & over']
    jdict[year_band]['300 & over']['per_tests'] =\
        jdict[year_band]['300 & over']['tests']/jdict[year_band]['300 & over']['total']

jstring = json.dumps(jdict, indent=4, sort_keys=True)

filenameg = 'again_indiv_scores.json' 
fnameg = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenameg)

with io.open(fnameg, 'w') as json_file:
    json_file.write(unicode(jstring))

print "Processing done"

Processing done


In [183]:
#this is just the json containing the run rate for test matches in a particular year
#honestly this could have been in a CSV, there's a consistent depth and consistent no. of fields for everything
#so there really isn't the need for a json anymore. Did the very same thing, five or six code blocks below when i converted
#this into a CSV

with open('final_test_run_wicket_rate.json', 'r') as f:
     test_runs = json.load(f)
        
jdict = defaultdict(lambda: defaultdict(float))

for year in test_runs['test_nations'].keys():
    jdict[year]['runs'] = test_runs['test_nations'][year]['runs']
    jdict[year]['run_rate'] = test_runs['test_nations'][year]['run_rate']
    jdict[year]['overs'] = test_runs['test_nations'][year]['overs']

jstring = json.dumps(jdict, indent=4, sort_keys=True)

filenameg = 'again_run_rate.json' 
fnameg = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenameg)

with io.open(fnameg, 'w') as json_file:
    json_file.write(unicode(jstring))

print "Processing done"



Processing done


In [185]:
#this is a json that tells us on average how many overs we see between sixes
#by this point I'd decided to concentrate on sixes as they represent a greater risk than 4s or boundaries
#if the no. of sixes is going up, we can kind of see whether t20 cricket had something to do with it

with open('final_test_four_six_rate.json', 'r') as f:
     test_sixes = json.load(f)
        
jdict = defaultdict(lambda: defaultdict(float))

for year in test_sixes['test_nations'].keys():
    jdict[year]['sixes'] = test_sixes['test_nations'][year]['sixes']
    jdict[year]['sixes_rate'] = test_sixes['test_nations'][year]['sixes_rate']
    jdict[year]['overs'] = test_sixes['test_nations'][year]['overs']

jstring = json.dumps(jdict, indent=4, sort_keys=True)

filenameg = 'again_sixes_rate.json' 
fnameg = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenameg)

with io.open(fnameg, 'w') as json_file:
    json_file.write(unicode(jstring))

print "Processing done"



Processing done


In [186]:
#this looks at the percentage of dismissals that are run outs

with open('final_test_run_out_rate.json', 'r') as f:
     test_run_outs = json.load(f)
        
jdict = defaultdict(lambda: defaultdict(float))

for year in test_run_outs['test_nations'].keys():
    jdict[year]['run_outs'] = test_run_outs['test_nations'][year]['run_outs']
    jdict[year]['run_out_rate'] = test_run_outs['test_nations'][year]['run_out_rate']
    jdict[year]['wickets'] = test_run_outs['test_nations'][year]['wickets']

jstring = json.dumps(jdict, indent=4, sort_keys=True)

filenameg = 'again_run_outs.json' 
fnameg = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenameg)

with io.open(fnameg, 'w') as json_file:
    json_file.write(unicode(jstring))

print "Processing done"

Processing done


In [187]:
#this looks at what percentage of overs bowled are maiden overs

with open('final_test_maidens_rate.json', 'r') as f:
     test_maidens = json.load(f)
        
jdict = defaultdict(lambda: defaultdict(float))

for year in test_maidens['test_nations'].keys():
    jdict[year]['maidens'] = test_maidens['test_nations'][year]['maidens']
    jdict[year]['maiden_rate'] = test_maidens['test_nations'][year]['maiden_rate']
    jdict[year]['overs'] = test_maidens['test_nations'][year]['overs']

jstring = json.dumps(jdict, indent=4, sort_keys=True)

filenameg = 'again_maidens.json' 
fnameg = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenameg)

with io.open(fnameg, 'w') as json_file:
    json_file.write(unicode(jstring))

print "Processing done"

Processing done


In [188]:
#this looks at the rate at which wickets are falling, how many overs on average do we see between batsmen getting out

with open('final_test_run_wicket_rate.json', 'r') as f:
     test_wickets = json.load(f)
        
jdict = defaultdict(lambda: defaultdict(float))

for year in test_wickets['test_nations'].keys():
    jdict[year]['wickets'] = test_wickets['test_nations'][year]['wickets']
    jdict[year]['wicket_rate'] = test_wickets['test_nations'][year]['wicket_rate']
    jdict[year]['overs'] = test_wickets['test_nations'][year]['overs']

jstring = json.dumps(jdict, indent=4, sort_keys=True)

filenameg = 'again_wicket_freq.json' 
fnameg = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenameg)

with io.open(fnameg, 'w') as json_file:
    json_file.write(unicode(jstring))

print "Processing done"

Processing done


In [189]:
#this looks at what percentage of matches in a year are draws

with open('test_df_draws_by_year.json', 'r') as f:
     test_df_draws = json.load(f)

jdict = defaultdict(lambda: defaultdict(float))

for year in test_df_draws.keys():
    if year not in ['1990','2017']:
        for result in test_df_draws[year].keys():
            jdict[year]['total_tests'] += test_df_draws[year][result]
            jdict[year]['draws'] = test_df_draws[year]['draw']
            jdict[year]['draw_percent'] = (jdict[year]['draws']/jdict[year]['total_tests'])*100
            
            #note that the draw percent is actually calculated as many times as there are 'result' in [year].keys
            #it's ok this time, as the result was still calculated correctly, but you might want to keep that out of the 
            #loop next time
        
jstring = json.dumps(jdict, indent=4, sort_keys=True)

filenameg = 'again_draw_percent.json' 
fnameg = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenameg)

with io.open(fnameg, 'w') as json_file:
    json_file.write(unicode(jstring))

print "Processing done"

Processing done


In [4]:
#this is a do over of a json file that we'd created earlier. what this does is aggregate the no. of double and triple centuries
#instead of putting them in separate categories

import json
import os
import io
from collections import defaultdict

with open('again_indiv_scores.json', 'r') as f:
     test_indiv_again = json.load(f)

jdict = defaultdict(lambda: defaultdict(float))

#this is for all invidiual scores 200 & over

for year_band in test_indiv_again.keys():
    for score_range in test_indiv_again[year_band].keys():
        jdict[year_band]['tests'] = test_indiv_again[year_band][score_range]['tests']
        jdict[year_band]['total'] += test_indiv_again[year_band][score_range]['total']
        jdict[year_band]['per_tests'] =\
            jdict[year_band]['tests']/jdict[year_band]['total']
            
jstring = json.dumps(jdict, indent=4, sort_keys=True)

filenameg = 'again_indiv_scores_v2.json' 
fnameg = os.path.join(os.path.dirname(os.path.realpath('__file__')), filenameg)

with io.open(fnameg, 'w') as json_file:
    json_file.write(unicode(jstring))

print "Processing done"

Processing done


In [17]:
#actually wanted to create the visualisations using json files, but was running out of time, so I converted the jsons to CSVs
#and used them instead

#ended up making static graphics at the end, instead of creating some inventive visualisation of the data

#note to self: if you're going to make something in d3, dont' start the day before the deadline!

import os
import pandas as pd

for filex in os.listdir(os.curdir):
    if filex.startswith('again_'):
        if filex not in ['again_test_totals.json','again_indiv_scores.json']:
            df=pd.read_json(filex)
            df2 = df.transpose()
            df2.index.name = 'year'
            filey = filex.rstrip('.json') + '.csv'
            df2.to_csv(filey)

print "Processing done"

Processing done


In [43]:
#this was a particularly complicated json file, so I ended up splitting these into four separate CSV files in order to make
#graphics for it

import io
import json
import csv

with open('again_test_totals.json', 'r') as f:
     test_tots_deux = json.load(f)

headings = ['score','percent','total']

for year_band in test_tots_deux.keys():
    fileg = 'again_test_tot_' + year_band + '.csv'
    with open(fileg, "w") as filem:
        wr = csv.writer(filem, delimiter = ',' , quotechar = '"',lineterminator='\n')
        wr.writerow(headings)
    for score_range in test_tots_deux[year_band].keys():
        row_list=[]
        row_list.append(score_range)
        row_list.append(test_tots_deux[year_band][score_range]['percent'])
        row_list.append(test_tots_deux[year_band][score_range]['total'])
        with open(fileg, "a") as filen:
            wrz = csv.writer(filen, delimiter = ',' , quotechar = '"',lineterminator='\n' )
            wrz.writerow(row_list)

filen.close()
print 'done and done'

done and done


In [ ]:
# THE END, phew! :)